# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Gender-dynamics" data-toc-modified-id="Gender-dynamics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Gender dynamics</a></div><div class="lev2 toc-item"><a href="#Tweet-data-prep" data-toc-modified-id="Tweet-data-prep-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Tweet data prep</a></div><div class="lev3 toc-item"><a href="#Load-the-tweets" data-toc-modified-id="Load-the-tweets-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Load the tweets</a></div><div class="lev2 toc-item"><a href="#Tweet-analysis" data-toc-modified-id="Tweet-analysis-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Tweet analysis</a></div><div class="lev3 toc-item"><a href="#First-and-last-tweet-in-dataset" data-toc-modified-id="First-and-last-tweet-in-dataset-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>First and last tweet in dataset</a></div><div class="lev3 toc-item"><a href="#Number-of-tweets-in-dataset" data-toc-modified-id="Number-of-tweets-in-dataset-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Number of tweets in dataset</a></div><div class="lev2 toc-item"><a href="#Tweeter-data-prep" data-toc-modified-id="Tweeter-data-prep-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Tweeter data prep</a></div><div class="lev2 toc-item"><a href="#Prepare-the-tweeter-data" data-toc-modified-id="Prepare-the-tweeter-data-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Prepare the tweeter data</a></div><div class="lev3 toc-item"><a href="#Load-user-lookup" data-toc-modified-id="Load-user-lookup-141"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Load user lookup</a></div><div class="lev3 toc-item"><a href="#Tweets-in-dataset-per-tweeter" data-toc-modified-id="Tweets-in-dataset-per-tweeter-142"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Tweets in dataset per tweeter</a></div><div class="lev3 toc-item"><a href="#Load-user-info" data-toc-modified-id="Load-user-info-143"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Load user info</a></div><div class="lev3 toc-item"><a href="#Remove-users-with-no-tweets-in-dataset" data-toc-modified-id="Remove-users-with-no-tweets-in-dataset-144"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>Remove users with no tweets in dataset</a></div><div class="lev2 toc-item"><a href="#Tweeter-analysis" data-toc-modified-id="Tweeter-analysis-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Tweeter analysis</a></div><div class="lev3 toc-item"><a href="#Gender" data-toc-modified-id="Gender-151"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Gender</a></div><div class="lev3 toc-item"><a href="#Summary" data-toc-modified-id="Summary-152"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Summary</a></div><div class="lev4 toc-item"><a href="#All" data-toc-modified-id="All-1521"><span class="toc-item-num">1.5.2.1&nbsp;&nbsp;</span>All</a></div><div class="lev4 toc-item"><a href="#Female" data-toc-modified-id="Female-1522"><span class="toc-item-num">1.5.2.2&nbsp;&nbsp;</span>Female</a></div><div class="lev4 toc-item"><a href="#Male" data-toc-modified-id="Male-1523"><span class="toc-item-num">1.5.2.3&nbsp;&nbsp;</span>Male</a></div><div class="lev3 toc-item"><a href="#Verified" data-toc-modified-id="Verified-153"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>Verified</a></div><div class="lev4 toc-item"><a href="#All" data-toc-modified-id="All-1531"><span class="toc-item-num">1.5.3.1&nbsp;&nbsp;</span>All</a></div><div class="lev4 toc-item"><a href="#Female" data-toc-modified-id="Female-1532"><span class="toc-item-num">1.5.3.2&nbsp;&nbsp;</span>Female</a></div><div class="lev4 toc-item"><a href="#Male" data-toc-modified-id="Male-1533"><span class="toc-item-num">1.5.3.3&nbsp;&nbsp;</span>Male</a></div><div class="lev2 toc-item"><a href="#Mention-data-prep" data-toc-modified-id="Mention-data-prep-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Mention data prep</a></div><div class="lev3 toc-item"><a href="#Load-mentions-from-tweets" data-toc-modified-id="Load-mentions-from-tweets-161"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Load mentions from tweets</a></div><div class="lev3 toc-item"><a href="#Add-gender-of-mentioner" data-toc-modified-id="Add-gender-of-mentioner-162"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>Add gender of mentioner</a></div><div class="lev3 toc-item"><a href="#Distinct-tweets" data-toc-modified-id="Distinct-tweets-163"><span class="toc-item-num">1.6.3&nbsp;&nbsp;</span>Distinct tweets</a></div><div class="lev3 toc-item"><a href="#Distinct-mentioned-users" data-toc-modified-id="Distinct-mentioned-users-164"><span class="toc-item-num">1.6.4&nbsp;&nbsp;</span>Distinct mentioned users</a></div><div class="lev3 toc-item"><a href="#Limit-to-mentions-of-journalists" data-toc-modified-id="Limit-to-mentions-of-journalists-165"><span class="toc-item-num">1.6.5&nbsp;&nbsp;</span>Limit to mentions of journalists</a></div><div class="lev3 toc-item"><a href="#Functions-for-summarizing-mentions-by-beltway-journalists" data-toc-modified-id="Functions-for-summarizing-mentions-by-beltway-journalists-166"><span class="toc-item-num">1.6.6&nbsp;&nbsp;</span>Functions for summarizing mentions by beltway journalists</a></div><div class="lev2 toc-item"><a href="#Mentioned-analysis" data-toc-modified-id="Mentioned-analysis-17"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Mentioned analysis</a></div><div class="lev3 toc-item"><a href="#Original-tweets-by-gender" data-toc-modified-id="Original-tweets-by-gender-171"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>Original tweets by gender</a></div><div class="lev3 toc-item"><a href="#Most-prolific-original-tweeters" data-toc-modified-id="Most-prolific-original-tweeters-172"><span class="toc-item-num">1.7.2&nbsp;&nbsp;</span>Most prolific original tweeters</a></div><div class="lev3 toc-item"><a href="#All-mentioned-by-beltway-journalists" data-toc-modified-id="All-mentioned-by-beltway-journalists-173"><span class="toc-item-num">1.7.3&nbsp;&nbsp;</span>All mentioned by beltway journalists</a></div><div class="lev4 toc-item"><a href="#All-mentioned-by-beltway-journalists-ordered-by-mentioning-count" data-toc-modified-id="All-mentioned-by-beltway-journalists-ordered-by-mentioning-count-1731"><span class="toc-item-num">1.7.3.1&nbsp;&nbsp;</span>All mentioned by beltway journalists ordered by mentioning count</a></div><div class="lev3 toc-item"><a href="#All-beltway-journalists-mentioned-by-beltway-journalists" data-toc-modified-id="All-beltway-journalists-mentioned-by-beltway-journalists-174"><span class="toc-item-num">1.7.4&nbsp;&nbsp;</span>All beltway journalists mentioned by beltway journalists</a></div><div class="lev4 toc-item"><a href="#All-beltway-journalists-mentioned-by-beltway-journalists-ordered-by-mention-count" data-toc-modified-id="All-beltway-journalists-mentioned-by-beltway-journalists-ordered-by-mention-count-1741"><span class="toc-item-num">1.7.4.1&nbsp;&nbsp;</span>All beltway journalists mentioned by beltway journalists ordered by mention count</a></div><div class="lev4 toc-item"><a href="#All-beltway-journalists-mentioned-by-beltway-journalists-ordered-by-mentioning-count" data-toc-modified-id="All-beltway-journalists-mentioned-by-beltway-journalists-ordered-by-mentioning-count-1742"><span class="toc-item-num">1.7.4.2&nbsp;&nbsp;</span>All beltway journalists mentioned by beltway journalists ordered by mentioning count</a></div><div class="lev4 toc-item"><a href="#Stats-of-all-beltway-journalists-mentioned-by-beltway-journalists" data-toc-modified-id="Stats-of-all-beltway-journalists-mentioned-by-beltway-journalists-1743"><span class="toc-item-num">1.7.4.3&nbsp;&nbsp;</span>Stats of all beltway journalists mentioned by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-mentioned-by-beltway-journalists" data-toc-modified-id="Top-100-journalists-mentioned-by-beltway-journalists-1744"><span class="toc-item-num">1.7.4.4&nbsp;&nbsp;</span>Top 100 journalists mentioned by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Female-beltway-journalists-mentioned-by-beltway-journalists" data-toc-modified-id="Female-beltway-journalists-mentioned-by-beltway-journalists-175"><span class="toc-item-num">1.7.5&nbsp;&nbsp;</span>Female beltway journalists mentioned by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-female-beltway-journalists-mentioned-by-beltway-journalists" data-toc-modified-id="Stats-of-female-beltway-journalists-mentioned-by-beltway-journalists-1751"><span class="toc-item-num">1.7.5.1&nbsp;&nbsp;</span>Stats of female beltway journalists mentioned by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Male-beltway-journalists-mentioned-by-beltway-journalists" data-toc-modified-id="Male-beltway-journalists-mentioned-by-beltway-journalists-176"><span class="toc-item-num">1.7.6&nbsp;&nbsp;</span>Male beltway journalists mentioned by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-male-beltway-journalists-mentioned-by-beltway-journalists" data-toc-modified-id="Stats-of-male-beltway-journalists-mentioned-by-beltway-journalists-1761"><span class="toc-item-num">1.7.6.1&nbsp;&nbsp;</span>Stats of male beltway journalists mentioned by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Beltway-journalists-mentioned-by-female-beltway-journalists" data-toc-modified-id="Beltway-journalists-mentioned-by-female-beltway-journalists-177"><span class="toc-item-num">1.7.7&nbsp;&nbsp;</span>Beltway journalists mentioned by female beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-mentioned-by-female-beltway-journalists" data-toc-modified-id="Top-100-journalists-mentioned-by-female-beltway-journalists-1771"><span class="toc-item-num">1.7.7.1&nbsp;&nbsp;</span>Top 100 journalists mentioned by female beltway journalists</a></div><div class="lev3 toc-item"><a href="#Beltway-journalists-mentioned-by-male-beltway-journalists" data-toc-modified-id="Beltway-journalists-mentioned-by-male-beltway-journalists-178"><span class="toc-item-num">1.7.8&nbsp;&nbsp;</span>Beltway journalists mentioned by male beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-mentioned-by-male-beltway-journalists" data-toc-modified-id="Top-100-journalists-mentioned-by-male-beltway-journalists-1781"><span class="toc-item-num">1.7.8.1&nbsp;&nbsp;</span>Top 100 journalists mentioned by male beltway journalists</a></div><div class="lev2 toc-item"><a href="#Retweet-data-prep" data-toc-modified-id="Retweet-data-prep-18"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Retweet data prep</a></div><div class="lev3 toc-item"><a href="#Load-retweets-from-tweets" data-toc-modified-id="Load-retweets-from-tweets-181"><span class="toc-item-num">1.8.1&nbsp;&nbsp;</span>Load retweets from tweets</a></div><div class="lev3 toc-item"><a href="#Add-gender-of-retweeter" data-toc-modified-id="Add-gender-of-retweeter-182"><span class="toc-item-num">1.8.2&nbsp;&nbsp;</span>Add gender of retweeter</a></div><div class="lev3 toc-item"><a href="#Distinct-retweeted-users" data-toc-modified-id="Distinct-retweeted-users-183"><span class="toc-item-num">1.8.3&nbsp;&nbsp;</span>Distinct retweeted users</a></div><div class="lev3 toc-item"><a href="#Limit-to-retweeted-journalists" data-toc-modified-id="Limit-to-retweeted-journalists-184"><span class="toc-item-num">1.8.4&nbsp;&nbsp;</span>Limit to retweeted journalists</a></div><div class="lev3 toc-item"><a href="#Functions-for-summarizing-retweets-by-beltway-journalists" data-toc-modified-id="Functions-for-summarizing-retweets-by-beltway-journalists-185"><span class="toc-item-num">1.8.5&nbsp;&nbsp;</span>Functions for summarizing retweets by beltway journalists</a></div><div class="lev2 toc-item"><a href="#Retweet-analysis" data-toc-modified-id="Retweet-analysis-19"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Retweet analysis</a></div><div class="lev3 toc-item"><a href="#Retweets-and-quotes-by-gender" data-toc-modified-id="Retweets-and-quotes-by-gender-191"><span class="toc-item-num">1.9.1&nbsp;&nbsp;</span>Retweets and quotes by gender</a></div><div class="lev3 toc-item"><a href="#Most-prolific-retweeters" data-toc-modified-id="Most-prolific-retweeters-192"><span class="toc-item-num">1.9.2&nbsp;&nbsp;</span>Most prolific retweeters</a></div><div class="lev3 toc-item"><a href="#All-retweeted-by-beltway-journalists" data-toc-modified-id="All-retweeted-by-beltway-journalists-193"><span class="toc-item-num">1.9.3&nbsp;&nbsp;</span>All retweeted by beltway journalists</a></div><div class="lev3 toc-item"><a href="#All-beltway-journalists-retweeted-by-beltway-journalists" data-toc-modified-id="All-beltway-journalists-retweeted-by-beltway-journalists-194"><span class="toc-item-num">1.9.4&nbsp;&nbsp;</span>All beltway journalists retweeted by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-beltway-journalists-retweeted-by-beltway-journalists" data-toc-modified-id="Stats-of-beltway-journalists-retweeted-by-beltway-journalists-1941"><span class="toc-item-num">1.9.4.1&nbsp;&nbsp;</span>Stats of beltway journalists retweeted by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-retweeted-by-beltway-journalists" data-toc-modified-id="Top-100-journalists-retweeted-by-beltway-journalists-1942"><span class="toc-item-num">1.9.4.2&nbsp;&nbsp;</span>Top 100 journalists retweeted by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Female-beltway-journalists-retweeted-by-beltway-journalists" data-toc-modified-id="Female-beltway-journalists-retweeted-by-beltway-journalists-195"><span class="toc-item-num">1.9.5&nbsp;&nbsp;</span>Female beltway journalists retweeted by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-female-beltway-journalists-retweeted-by-beltway-journalists" data-toc-modified-id="Stats-of-female-beltway-journalists-retweeted-by-beltway-journalists-1951"><span class="toc-item-num">1.9.5.1&nbsp;&nbsp;</span>Stats of female beltway journalists retweeted by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Male-beltway-journalists-retweeted-by-beltway-journalists" data-toc-modified-id="Male-beltway-journalists-retweeted-by-beltway-journalists-196"><span class="toc-item-num">1.9.6&nbsp;&nbsp;</span>Male beltway journalists retweeted by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-male-beltway-journalists-retweeted-by-beltway-journalists" data-toc-modified-id="Stats-of-male-beltway-journalists-retweeted-by-beltway-journalists-1961"><span class="toc-item-num">1.9.6.1&nbsp;&nbsp;</span>Stats of male beltway journalists retweeted by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Beltway-journalists-retweeted-by-female-beltway-journalists" data-toc-modified-id="Beltway-journalists-retweeted-by-female-beltway-journalists-197"><span class="toc-item-num">1.9.7&nbsp;&nbsp;</span>Beltway journalists retweeted by female beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-retweeted-by-female-beltway-journalists" data-toc-modified-id="Top-100-journalists-retweeted-by-female-beltway-journalists-1971"><span class="toc-item-num">1.9.7.1&nbsp;&nbsp;</span>Top 100 journalists retweeted by female beltway journalists</a></div><div class="lev3 toc-item"><a href="#Beltway-journalists-retweeted-by-male-beltway-journalists" data-toc-modified-id="Beltway-journalists-retweeted-by-male-beltway-journalists-198"><span class="toc-item-num">1.9.8&nbsp;&nbsp;</span>Beltway journalists retweeted by male beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-retweeted-by-male-beltway-journalists" data-toc-modified-id="Top-100-journalists-retweeted-by-male-beltway-journalists-1981"><span class="toc-item-num">1.9.8.1&nbsp;&nbsp;</span>Top 100 journalists retweeted by male beltway journalists</a></div><div class="lev2 toc-item"><a href="#Reply-data-prep" data-toc-modified-id="Reply-data-prep-110"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Reply data prep</a></div><div class="lev3 toc-item"><a href="#Load-replies-from-tweets" data-toc-modified-id="Load-replies-from-tweets-1101"><span class="toc-item-num">1.10.1&nbsp;&nbsp;</span>Load replies from tweets</a></div><div class="lev3 toc-item"><a href="#Add-gender-of-replier" data-toc-modified-id="Add-gender-of-replier-1102"><span class="toc-item-num">1.10.2&nbsp;&nbsp;</span>Add gender of replier</a></div><div class="lev3 toc-item"><a href="#Distinct-replied-to-users" data-toc-modified-id="Distinct-replied-to-users-1103"><span class="toc-item-num">1.10.3&nbsp;&nbsp;</span>Distinct replied to users</a></div><div class="lev3 toc-item"><a href="#Limit-to-beltway-journalists" data-toc-modified-id="Limit-to-beltway-journalists-1104"><span class="toc-item-num">1.10.4&nbsp;&nbsp;</span>Limit to beltway journalists</a></div><div class="lev3 toc-item"><a href="#Functions-for-summarizing-replies-by-beltway-journalists" data-toc-modified-id="Functions-for-summarizing-replies-by-beltway-journalists-1105"><span class="toc-item-num">1.10.5&nbsp;&nbsp;</span>Functions for summarizing replies by beltway journalists</a></div><div class="lev2 toc-item"><a href="#Reply-analysis" data-toc-modified-id="Reply-analysis-111"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Reply analysis</a></div><div class="lev3 toc-item"><a href="#Replies-by-gender" data-toc-modified-id="Replies-by-gender-1111"><span class="toc-item-num">1.11.1&nbsp;&nbsp;</span>Replies by gender</a></div><div class="lev3 toc-item"><a href="#Most-prolific-repliers" data-toc-modified-id="Most-prolific-repliers-1112"><span class="toc-item-num">1.11.2&nbsp;&nbsp;</span>Most prolific repliers</a></div><div class="lev3 toc-item"><a href="#All-replied-to-by-beltway-journalists" data-toc-modified-id="All-replied-to-by-beltway-journalists-1113"><span class="toc-item-num">1.11.3&nbsp;&nbsp;</span>All replied to by beltway journalists</a></div><div class="lev3 toc-item"><a href="#All-beltway-journalists-replied-to-by-beltway-journalists" data-toc-modified-id="All-beltway-journalists-replied-to-by-beltway-journalists-1114"><span class="toc-item-num">1.11.4&nbsp;&nbsp;</span>All beltway journalists replied to by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-beltway-journalists-replied-to-by-beltway-journalists" data-toc-modified-id="Stats-of-beltway-journalists-replied-to-by-beltway-journalists-11141"><span class="toc-item-num">1.11.4.1&nbsp;&nbsp;</span>Stats of beltway journalists replied to by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-replied-to-by-beltway-journalists" data-toc-modified-id="Top-100-journalists-replied-to-by-beltway-journalists-11142"><span class="toc-item-num">1.11.4.2&nbsp;&nbsp;</span>Top 100 journalists replied to by beltway journalists</a></div><div class="lev1 toc-item"><a href="#Female-beltway-journalists-replied-to-by-beltway-journalists" data-toc-modified-id="Female-beltway-journalists-replied-to-by-beltway-journalists-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Female beltway journalists replied to by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-female-beltway-journalists-replied-to-by-beltway-journalists" data-toc-modified-id="Stats-of-female-beltway-journalists-replied-to-by-beltway-journalists-2001"><span class="toc-item-num">2.0.0.1&nbsp;&nbsp;</span>Stats of female beltway journalists replied to by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Male-beltway-journalists-replied-to-by-beltway-journalists" data-toc-modified-id="Male-beltway-journalists-replied-to-by-beltway-journalists-201"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Male beltway journalists replied to by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-male-beltway-journalists-replied-to-by-beltway-journalists" data-toc-modified-id="Stats-of-male-beltway-journalists-replied-to-by-beltway-journalists-2011"><span class="toc-item-num">2.0.1.1&nbsp;&nbsp;</span>Stats of male beltway journalists replied to by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Beltway-journalists-replied-to-by-female-beltway-journalists" data-toc-modified-id="Beltway-journalists-replied-to-by-female-beltway-journalists-202"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Beltway journalists replied to by female beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-replied-to-by-female-beltway-journalists" data-toc-modified-id="Top-100-journalists-replied-to-by-female-beltway-journalists-2021"><span class="toc-item-num">2.0.2.1&nbsp;&nbsp;</span>Top 100 journalists replied to by female beltway journalists</a></div><div class="lev3 toc-item"><a href="#Beltway-journalists-replied-to-by-male-beltway-journalists" data-toc-modified-id="Beltway-journalists-replied-to-by-male-beltway-journalists-203"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>Beltway journalists replied to by male beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-replied-to-by-male-beltway-journalists" data-toc-modified-id="Top-100-journalists-replied-to-by-male-beltway-journalists-2031"><span class="toc-item-num">2.0.3.1&nbsp;&nbsp;</span>Top 100 journalists replied to by male beltway journalists</a></div><div class="lev2 toc-item"><a href="#Following-data-prep" data-toc-modified-id="Following-data-prep-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Following data prep</a></div><div class="lev3 toc-item"><a href="#Load-following" data-toc-modified-id="Load-following-211"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Load following</a></div><div class="lev3 toc-item"><a href="#Load-followed-users" data-toc-modified-id="Load-followed-users-212"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Load followed users</a></div><div class="lev3 toc-item"><a href="#Limit-to-beltway-journalists" data-toc-modified-id="Limit-to-beltway-journalists-213"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Limit to beltway journalists</a></div><div class="lev3 toc-item"><a href="#Functions-for-summarizing-following-by-beltway-journalists" data-toc-modified-id="Functions-for-summarizing-following-by-beltway-journalists-214"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Functions for summarizing following by beltway journalists</a></div><div class="lev2 toc-item"><a href="#Following-analysis" data-toc-modified-id="Following-analysis-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Following analysis</a></div><div class="lev3 toc-item"><a href="#All-followed-by-beltway-journalists" data-toc-modified-id="All-followed-by-beltway-journalists-221"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>All followed by beltway journalists</a></div><div class="lev3 toc-item"><a href="#All-beltway-journalists-followed-by-beltway-journalists" data-toc-modified-id="All-beltway-journalists-followed-by-beltway-journalists-222"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>All beltway journalists followed by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-beltway-journalists-followed-by-beltway-journalists" data-toc-modified-id="Stats-of-beltway-journalists-followed-by-beltway-journalists-2221"><span class="toc-item-num">2.2.2.1&nbsp;&nbsp;</span>Stats of beltway journalists followed by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-followed-by-beltway-journalists" data-toc-modified-id="Top-100-journalists-followed-by-beltway-journalists-2222"><span class="toc-item-num">2.2.2.2&nbsp;&nbsp;</span>Top 100 journalists followed by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Female-beltway-journalists-followed-by-beltway-journalists" data-toc-modified-id="Female-beltway-journalists-followed-by-beltway-journalists-223"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Female beltway journalists followed by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-female-beltway-journalists-followed-by-beltway-journalists" data-toc-modified-id="Stats-of-female-beltway-journalists-followed-by-beltway-journalists-2231"><span class="toc-item-num">2.2.3.1&nbsp;&nbsp;</span>Stats of female beltway journalists followed by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Male-beltway-journalists-followed-by-beltway-journalists" data-toc-modified-id="Male-beltway-journalists-followed-by-beltway-journalists-224"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Male beltway journalists followed by beltway journalists</a></div><div class="lev4 toc-item"><a href="#Stats-of-male-beltway-journalists-followed-by-beltway-journalists" data-toc-modified-id="Stats-of-male-beltway-journalists-followed-by-beltway-journalists-2241"><span class="toc-item-num">2.2.4.1&nbsp;&nbsp;</span>Stats of male beltway journalists followed by beltway journalists</a></div><div class="lev3 toc-item"><a href="#Beltway-journalists-followed-by-female-beltway-journalists" data-toc-modified-id="Beltway-journalists-followed-by-female-beltway-journalists-225"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>Beltway journalists followed by female beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-followed-by-female-beltway-journalists" data-toc-modified-id="Top-100-journalists-followed-by-female-beltway-journalists-2251"><span class="toc-item-num">2.2.5.1&nbsp;&nbsp;</span>Top 100 journalists followed by female beltway journalists</a></div><div class="lev3 toc-item"><a href="#Beltway-journalists-followed-by-male-beltway-journalists" data-toc-modified-id="Beltway-journalists-followed-by-male-beltway-journalists-226"><span class="toc-item-num">2.2.6&nbsp;&nbsp;</span>Beltway journalists followed by male beltway journalists</a></div><div class="lev4 toc-item"><a href="#Top-100-journalists-followed-by-male-beltway-journalists" data-toc-modified-id="Top-100-journalists-followed-by-male-beltway-journalists-2261"><span class="toc-item-num">2.2.6.1&nbsp;&nbsp;</span>Top 100 journalists followed by male beltway journalists</a></div><div class="lev2 toc-item"><a href="#Merge-all" data-toc-modified-id="Merge-all-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Merge all</a></div><div class="lev3 toc-item"><a href="#Describe-a-row" data-toc-modified-id="Describe-a-row-231"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Describe a row</a></div>

# Gender dynamics

## Tweet data prep

### Load the tweets

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import logging
from dateutil.parser import parse as date_parse
from utils import load_tweet_df, tweet_type
import matplotlib.pyplot as plt


logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Set float format so doesn't display scientific notation
pd.options.display.float_format = '{:20,.2f}'.format

def tweet_transform(tweet):
    return {
        'tweet_id': tweet['id_str'], 
        'tweet_created_at': date_parse(tweet['created_at']),
        'user_id': tweet['user']['id_str'],
        'screen_name': tweet['user']['screen_name'],
        'tweet_type': tweet_type(tweet)
    }

tweet_df = load_tweet_df(tweet_transform, ['tweet_id', 'user_id', 'screen_name', 'tweet_created_at', 'tweet_type'], dedupe_columns=['tweet_id'])
tweet_df.count()

INFO:root:Loading from tweets/41feff28312c433ab004cd822212f4c2_001.json.gz
DEBUG:root:Loaded 50000
DEBUG:root:Loaded 100000
DEBUG:root:Loaded 150000
DEBUG:root:Loaded 200000
DEBUG:root:Loaded 250000
DEBUG:root:Loaded 300000
INFO:root:Loading from tweets/642bf140607547cb9d4c6b1fc49772aa_001.json.gz
DEBUG:root:Loaded 350000
DEBUG:root:Loaded 400000
DEBUG:root:Loaded 450000
DEBUG:root:Loaded 500000
DEBUG:root:Loaded 550000
INFO:root:Loading from tweets/9f7ed17c16a1494c8690b4053609539d_001.json.gz
DEBUG:root:Loaded 600000
DEBUG:root:Loaded 650000
DEBUG:root:Loaded 700000
DEBUG:root:Loaded 750000
DEBUG:root:Loaded 800000


tweet_id            817136
user_id             817136
screen_name         817136
tweet_created_at    817136
tweet_type          817136
dtype: int64

In [2]:
tweet_df.head()

,tweet_id,user_id,screen_name,tweet_created_at,tweet_type
0,875028934177542144,15731368,HowardKurtz,2017-06-14 16:35:29+00:00,original
1,875012559228874752,15731368,HowardKurtz,2017-06-14 15:30:25+00:00,retweet
2,875012374343929862,15731368,HowardKurtz,2017-06-14 15:29:41+00:00,original
3,875010759759101952,15731368,HowardKurtz,2017-06-14 15:23:16+00:00,retweet
4,875010163496939520,15731368,HowardKurtz,2017-06-14 15:20:54+00:00,retweet


## Tweet analysis

### First and last tweet in dataset

In [3]:
tweet_df.tweet_created_at.min()

Timestamp('2017-06-01 04:00:01+0000', tz='UTC')

In [4]:
tweet_df.tweet_created_at.max()

Timestamp('2017-08-01 03:59:58+0000', tz='UTC')

### Number of tweets in dataset

In [5]:
pd.DataFrame({'count':tweet_df.tweet_type.value_counts(), 
              'percentage':tweet_df.tweet_type.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
retweet,345266,42.3%
original,233926,28.6%
reply,126254,15.5%
quote,111690,13.7%


## Tweeter data prep

## Prepare the tweeter data
This comes from the following sources:
1. User lookup: These are lists of users exported from SFM. These are the final set of beltway journalists. Accounts that were suspended or deleted have been removed from this list. Also, this list will include users that did not tweet (i.e., have no tweets in dataset).
2. Tweets in the dataset: Used to generate tweet counts per tweeter. However, since some beltway journalists may not have tweeted, this may be a subset of the user lookup. Also, it may include the tweets of some users that were later excluded because their accounts were suspended or deleted or determined to not be beltway journalists.
3. User info lookup: Information on users that was manually coded in the beltway journalist spreadsheet or looked up from Twitter's API. This includes some accounts that were excluded from data collection for various reasons such as working for a foreign news organization or no longer working as a beltway journalist. Thus, these are a superset of the user lookup.

Thus, the tweeter data should include tweet and user info data only from users in the user lookup.

### Load user lookup

In [6]:
user_lookup_filepaths = ('lookups/senate_press_lookup.csv',
                         'lookups/periodical_press_lookup.csv',
                         'lookups/radio_and_television_lookup.csv')
user_lookup_df = pd.concat((pd.read_csv(user_lookup_filepath, usecols=['Uid', 'Token'], dtype={'Uid': str}) for user_lookup_filepath in user_lookup_filepaths))
user_lookup_df.set_index('Uid', inplace=True)
user_lookup_df.rename(columns={'Token': 'screen_name'}, inplace=True)
user_lookup_df.index.names = ['user_id']
# Some users may be in multiple lists, so need to drop duplicates
user_lookup_df = user_lookup_df[~user_lookup_df.index.duplicated()]

user_lookup_df.count()

screen_name    2487
dtype: int64

In [7]:
user_lookup_df.head()

,screen_name
user_id,
23455653,abettel
33919343,AshleyRParker
18580432,b_fung
399225358,b_muzz
18834692,becca_milfeld


### Tweets in dataset per tweeter

In [8]:
user_tweet_count_df = tweet_df[['user_id', 'tweet_type']].groupby(['user_id', 'tweet_type']).size().unstack()
user_tweet_count_df.fillna(0, inplace=True)
user_tweet_count_df['tweets_in_dataset'] = user_tweet_count_df.original + user_tweet_count_df.quote + user_tweet_count_df.reply + user_tweet_count_df.retweet
user_tweet_count_df.count()

tweet_type
original             2292
quote                2292
reply                2292
retweet              2292
tweets_in_dataset    2292
dtype: int64

In [9]:
user_tweet_count_df.head()

tweet_type,original,quote,reply,retweet,tweets_in_dataset
user_id,,,,,
1001991865,13.00,3.00,1.00,31.00,48.00
1002229862,48.00,20.00,3.00,118.00,189.00
100270054,1.00,0.00,0.00,0.00,1.00
100802089,4.00,7.00,12.00,17.00,40.00
100860790,102.00,26.00,4.00,166.00,298.00


### Load user info

In [10]:
user_info_df = pd.read_csv('source_data/user_info_lookup.csv', names=['user_id', 'name', 'organization', 'position',
                                            'gender', 'followers_count', 'following_count', 'tweet_count',
                                            'user_created_at', 'verified', 'protected'],
                          dtype={'user_id': str}).set_index(['user_id'])
user_info_df.count()

name               2506
organization       2477
position           2503
gender             2505
followers_count    2506
following_count    2506
tweet_count        2506
user_created_at    2506
verified           2506
protected          2506
dtype: int64

In [11]:
user_info_df.head()

,name,organization,position,gender,followers_count,following_count,tweet_count,user_created_at,verified,protected
user_id,,,,,,,,,,
20711445,"Glinski, Nina",NaN,Freelance Reporter,F,963,507,909,Thu Feb 12 20:00:53 +0000 2009,False,False
258917371,"Enders, David",NaN,Journalist,M,1444,484,6296,Mon Feb 28 19:52:03 +0000 2011,True,False
297046834,"Barakat, Matthew",Associated Press,Northern Virginia Correspondent,M,759,352,631,Wed May 11 20:55:24 +0000 2011,True,False
455585786,"Atkins, Kimberly",Boston Herald,Chief Washington Reporter/Columnist,F,2944,2691,6277,Thu Jan 05 08:26:46 +0000 2012,True,False
42584840,"Vlahou, Toula",CQ Roll Call,Editor & Podcast Producer,F,2703,201,6366,Tue May 26 07:41:38 +0000 2009,False,False


In [12]:
user_summary_df = user_lookup_df.join((user_info_df, user_tweet_count_df), how='left')
# Fill Nans
user_summary_df['organization'].fillna('', inplace=True)
user_summary_df['original'].fillna(0, inplace=True)
user_summary_df['quote'].fillna(0, inplace=True)
user_summary_df['reply'].fillna(0, inplace=True)
user_summary_df['retweet'].fillna(0, inplace=True)
user_summary_df['tweets_in_dataset'].fillna(0, inplace=True)
user_summary_df.count()

screen_name          2487
name                 2487
organization         2487
position             2484
gender               2486
followers_count      2487
following_count      2487
tweet_count          2487
user_created_at      2487
verified             2487
protected            2487
original             2487
quote                2487
reply                2487
retweet              2487
tweets_in_dataset    2487
dtype: int64

In [13]:
user_summary_df.head()

,screen_name,name,organization,position,gender,followers_count,following_count,tweet_count,user_created_at,verified,protected,original,quote,reply,retweet,tweets_in_dataset
user_id,,,,,,,,,,,,,,,,
23455653,abettel,"Bettelheim, Adriel",Politico,Health Care Editor,F,2664,1055,15990,Mon Mar 09 16:32:20 +0000 2009,True,False,289.00,12.00,6.00,52.00,359.00
33919343,AshleyRParker,"Parker, Ashley",Washington Post,White House Reporter,F,122382,2342,12433,Tue Apr 21 14:28:57 +0000 2009,True,False,172.00,67.00,11.00,120.00,370.00
18580432,b_fung,"Fung, Brian",Washington Post,Tech Reporter,M,16558,2062,44799,Sat Jan 03 15:15:57 +0000 2009,True,False,257.00,85.00,205.00,82.00,629.00
399225358,b_muzz,"Murray, Brendan",Bloomberg News,"Managing Editor, U.S. Economy",M,624,382,360,Thu Oct 27 05:34:05 +0000 2011,True,False,3.00,0.00,0.00,5.00,8.00
18834692,becca_milfeld,"Milfeld, Becca",Agence France-Presse,English Desk Editor and Journalist,F,483,993,1484,Sat Jan 10 13:58:43 +0000 2009,False,False,3.00,14.00,0.00,7.00,24.00


### Remove users with no tweets in dataset

In [14]:
user_summary_df[user_summary_df.tweets_in_dataset == 0].count()

screen_name          195
name                 195
organization         195
position             195
gender               194
followers_count      195
following_count      195
tweet_count          195
user_created_at      195
verified             195
protected            195
original             195
quote                195
reply                195
retweet              195
tweets_in_dataset    195
dtype: int64

In [15]:
user_summary_df = user_summary_df[user_summary_df.tweets_in_dataset != 0]
user_summary_df.count()

screen_name          2292
name                 2292
organization         2292
position             2289
gender               2292
followers_count      2292
following_count      2292
tweet_count          2292
user_created_at      2292
verified             2292
protected            2292
original             2292
quote                2292
reply                2292
retweet              2292
tweets_in_dataset    2292
dtype: int64

## Tweeter analysis

### Gender

In [16]:
pd.DataFrame({'count':user_summary_df.gender.value_counts(), 'percentage':user_summary_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
M,1299,56.7%
F,993,43.3%


### Summary

* 25%, 50%, 75% are the percentiles. (Min is equivalent to 0%. Max is equivalent to 100%. 50% is the median.)
* std is standard deviation, normalized by N-1.

#### All

In [17]:
user_summary_df[['followers_count', 'following_count', 'tweet_count', 'original', 'quote', 'reply', 'retweet', 'tweets_in_dataset']].describe()

,followers_count,following_count,tweet_count,original,quote,reply,retweet,tweets_in_dataset
count,"2,292.00","2,292.00","2,292.00","2,292.00","2,292.00","2,292.00","2,292.00","2,292.00"
mean,"16,467.62","1,444.83","9,619.69",102.06,48.73,55.08,150.64,356.52
std,"91,886.90","3,003.00","16,618.09",169.43,135.90,249.18,585.08,833.76
min,6.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
25%,831.75,505.75,"1,449.50",10.00,1.00,1.00,8.00,32.00
50%,"2,419.50",998.50,"4,211.50",41.00,9.00,5.00,39.00,122.00
75%,"7,348.75","1,713.50","10,817.25",124.25,43.00,30.00,129.00,375.00
max,"2,176,578.00","96,194.00","208,763.00","2,693.00","3,069.00","9,033.00","21,524.00","21,547.00"


#### Female

In [18]:
user_summary_df[user_summary_df.gender == 'F'][['followers_count', 'following_count', 'tweet_count', 'original', 'quote', 'reply', 'retweet', 'tweets_in_dataset']].describe()

,followers_count,following_count,tweet_count,original,quote,reply,retweet,tweets_in_dataset
count,993.00,993.00,993.00,993.00,993.00,993.00,993.00,993.00
mean,"11,609.53","1,314.07","7,498.74",83.84,39.27,32.06,135.55,290.72
std,"65,563.72","1,250.56","11,312.72",124.86,135.05,94.73,724.92,833.07
min,6.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00
25%,825.00,567.00,"1,393.00",8.00,1.00,1.00,9.00,32.00
50%,"2,327.00","1,034.00","4,055.00",39.00,9.00,4.00,37.00,111.00
75%,"6,340.00","1,659.00","8,983.00",111.00,33.00,21.00,115.00,314.00
max,"1,388,543.00","18,197.00","118,713.00","1,440.00","3,069.00","1,458.00","21,524.00","21,547.00"


#### Male

In [19]:
user_summary_df[user_summary_df.gender == 'M'][['followers_count', 'following_count', 'tweet_count', 'original', 'quote', 'reply', 'retweet', 'tweets_in_dataset']].describe()

,followers_count,following_count,tweet_count,original,quote,reply,retweet,tweets_in_dataset
count,"1,299.00","1,299.00","1,299.00","1,299.00","1,299.00","1,299.00","1,299.00","1,299.00"
mean,"20,181.31","1,544.78","11,241.02",115.99,55.96,72.69,162.17,406.81
std,"107,635.37","3,833.89","19,584.46",195.72,136.16,319.41,449.75,831.10
min,10.00,0.00,5.00,0.00,0.00,0.00,0.00,1.00
25%,857.50,472.00,"1,477.00",12.00,0.00,1.00,6.00,33.00
50%,"2,498.00",953.00,"4,401.00",44.00,9.00,6.00,40.00,131.00
75%,"8,341.50","1,763.00","12,584.50",140.00,50.50,38.50,142.00,428.00
max,"2,176,578.00","96,194.00","208,763.00","2,693.00","1,955.00","9,033.00","7,528.00","11,432.00"


### Verified

#### All

In [20]:
pd.DataFrame({'count':user_summary_df.verified.value_counts(), 'percentage':user_summary_df.verified.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
True,1240,54.1%
False,1052,45.9%


#### Female

In [21]:
pd.DataFrame({'count':user_summary_df[user_summary_df.gender == 'F'].verified.value_counts(), 'percentage':user_summary_df[user_summary_df.gender == 'F'].verified.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
True,512,51.6%
False,481,48.4%


#### Male

In [22]:
pd.DataFrame({'count':user_summary_df[user_summary_df.gender == 'M'].verified.value_counts(), 'percentage':user_summary_df[user_summary_df.gender == 'M'].verified.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
True,728,56.0%
False,571,44.0%


## Mention data prep

### Load mentions from tweets
Including original tweets only

In [23]:
%matplotlib inline
import pandas as pd
import numpy as np
import logging
from dateutil.parser import parse as date_parse
from utils import load_tweet_df, tweet_type
import matplotlib.pyplot as plt


logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Set float format so doesn't display scientific notation
pd.options.display.float_format = '{:20,.2f}'.format

# Simply the tweet on load
def mention_transform(tweet):
    mentions = []
    if tweet_type(tweet) == 'original':
        for mention in tweet.get('entities', {}).get('user_mentions', []):
            mentions.append({
                'tweet_id': tweet['id_str'],
                'user_id': tweet['user']['id_str'],
                'screen_name': tweet['user']['screen_name'],
                'mention_user_id': mention['id_str'],
                'mention_screen_name': mention['screen_name'],
                'tweet_created_at': date_parse(tweet['created_at'])
            })
    return mentions

base_mention_df = load_tweet_df(mention_transform, ['tweet_id', 'user_id', 'screen_name', 'mention_user_id',
                                           'mention_screen_name', 'tweet_created_at'], 
                           dedupe_columns=['tweet_id', 'mention_user_id'])
base_mention_df.count()

INFO:root:Loading from tweets/41feff28312c433ab004cd822212f4c2_001.json.gz
DEBUG:root:Loaded 50000
DEBUG:root:Loaded 100000
DEBUG:root:Loaded 150000
DEBUG:root:Loaded 200000
DEBUG:root:Loaded 250000
DEBUG:root:Loaded 300000
INFO:root:Loading from tweets/642bf140607547cb9d4c6b1fc49772aa_001.json.gz
DEBUG:root:Loaded 350000
DEBUG:root:Loaded 400000
DEBUG:root:Loaded 450000
DEBUG:root:Loaded 500000
DEBUG:root:Loaded 550000
INFO:root:Loading from tweets/9f7ed17c16a1494c8690b4053609539d_001.json.gz
DEBUG:root:Loaded 600000
DEBUG:root:Loaded 650000
DEBUG:root:Loaded 700000
DEBUG:root:Loaded 750000
DEBUG:root:Loaded 800000


tweet_id               118210
user_id                118210
screen_name            118210
mention_user_id        118210
mention_screen_name    118210
tweet_created_at       118210
dtype: int64

In [24]:
base_mention_df.head()

,tweet_id,user_id,screen_name,mention_user_id,mention_screen_name,tweet_created_at
0,874695069584027648,15731368,HowardKurtz,110445334,megynkelly,2017-06-13 18:28:50+00:00
1,874695069584027648,15731368,HowardKurtz,171154131,HappeningNow,2017-06-13 18:28:50+00:00
2,874652287809064960,15731368,HowardKurtz,110445334,megynkelly,2017-06-13 15:38:50+00:00
3,874281339150884864,15731368,HowardKurtz,486907980,CarleyShimkus,2017-06-12 15:04:49+00:00
4,874265555552882688,15731368,HowardKurtz,2263438704,GillianHTurner,2017-06-12 14:02:06+00:00


### Add gender of mentioner

In [25]:
mention_df = base_mention_df.join(user_summary_df['gender'], on='user_id')
mention_df.count()

tweet_id               118210
user_id                118210
screen_name            118210
mention_user_id        118210
mention_screen_name    118210
tweet_created_at       118210
gender                 118210
dtype: int64

### Distinct tweets

In [26]:
mention_df['tweet_id'].unique().size

84942

### Distinct mentioned users

In [27]:
mention_df['mention_user_id'].unique().size

17730

### Limit to mentions of journalists

In [28]:
journalists_mention_df = mention_df.join(user_summary_df['gender'], how='inner', on='mention_user_id', rsuffix='_mention')
journalists_mention_df.rename(columns = {'gender_mention': 'mention_gender'}, inplace=True)
journalists_mention_df.count()

tweet_id               14298
user_id                14298
screen_name            14298
mention_user_id        14298
mention_screen_name    14298
tweet_created_at       14298
gender                 14298
mention_gender         14298
dtype: int64

In [29]:
journalists_mention_df.head()

,tweet_id,user_id,screen_name,mention_user_id,mention_screen_name,tweet_created_at,gender,mention_gender
9,873955901484486656,15731368,HowardKurtz,16157855,edhenry,2017-06-11 17:31:38+00:00,M,M
14,873877145117962241,15731368,HowardKurtz,16157855,edhenry,2017-06-11 12:18:41+00:00,M,M
6414,874439120344535041,19455864,finnygo,16157855,edhenry,2017-06-13 01:31:47+00:00,M,M
23175,879021641141022720,15731368,HowardKurtz,16157855,edhenry,2017-06-25 17:01:05+00:00,M,M
23178,878956408657633280,15731368,HowardKurtz,16157855,edhenry,2017-06-25 12:41:52+00:00,M,M


### Functions for summarizing mentions by beltway journalists

In [30]:
# Gender of beltway journalists mentioned by beltway journalists
def journalist_mention_gender_summary(mention_df):
    return pd.DataFrame({'count': mention_df.mention_gender.value_counts(), 
                  'percentage': mention_df.mention_gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

def journalist_mention_summary(mention_df):
    # Mention count
    mention_count_df = pd.DataFrame(mention_df.mention_user_id.value_counts().rename('mention_count'))

    # Mentioning users. That is, the number of unique users mentioning each user.
    mention_user_id_per_user_df = mention_df[['mention_user_id', 'user_id']].drop_duplicates()
    mentioning_user_count_df = pd.DataFrame(mention_user_id_per_user_df.groupby('mention_user_id').size(), columns=['mentioning_count'])
    mentioning_user_count_df.index.name = 'user_id'

    # Join with user summary
    journalist_mention_summary_df = user_summary_df.join([mention_count_df, mentioning_user_count_df])
    journalist_mention_summary_df.fillna(0, inplace=True)
    journalist_mention_summary_df = journalist_mention_summary_df.sort_values(['mention_count', 'mentioning_count', 'followers_count'], ascending=False)
    return journalist_mention_summary_df

# Gender of top journalists mentioned by beltway journalists
def top_journalist_mention_gender_summary(mention_summary_df, mentioning_count_threshold=0, head=100):
    top_mention_summary_df = mention_summary_df[mention_summary_df.mentioning_count > mentioning_count_threshold].head(head)
    return pd.DataFrame({'count': top_mention_summary_df.gender.value_counts(), 
                  'percentage': top_mention_summary_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})


# Fields for displaying journalist mention summaries
journalist_mention_summary_fields = ['screen_name', 'name', 'organization', 'gender', 'followers_count', 'mention_count', 'mentioning_count']


## Mentioned analysis
*Note that for each of these, the complete list is being written to CSV in the output directory.*


### Original tweets by gender

In [31]:
original_tweets_by_gender_df = user_summary_df[['gender', 'original']].groupby('gender').sum()
original_tweets_by_gender_df['percentage'] = original_tweets_by_gender_df.original.div(user_summary_df.original.sum()).mul(100).round(1).astype(str) + '%'
original_tweets_by_gender_df

,original,percentage
gender,,
F,"83,251.00",35.6%
M,"150,675.00",64.4%


### Most prolific original tweeters

In [32]:
user_summary_df[['screen_name', 'name', 'organization', 'gender', 'followers_count', 'tweet_count', 'original', 'tweets_in_dataset']].sort_values(['original'], ascending=False).head(25)

,screen_name,name,organization,gender,followers_count,tweet_count,original,tweets_in_dataset
user_id,,,,,,,,
16187637,ChadPergram,"Pergram, Chad",Fox News,M,59305,61461,"2,693.00","2,693.00"
31127446,markknoller,"Knoller, Mark",CBS News,M,301474,115132,"1,858.00","2,089.00"
16459325,ryanbeckwith,"Beckwith, Ryan Teague",Time Magazine,M,20947,92203,"1,534.00","5,187.00"
19580890,LeeCamp,"Camp, Lee",RTTV America,M,67601,52051,"1,517.00","3,708.00"
18825339,CahnEmily,"Cahn, Emily",Mic,F,16980,100803,"1,440.00","8,196.00"
593813785,DonnaYoungDC,"Young, Donna",S&P Global Market Intelligence,F,5894,49967,"1,332.00","4,414.00"
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,148143,"1,316.00","5,078.00"
21316253,ZekeJMiller,"Miller, Zeke J.",Time Magazine,M,198517,161148,"1,271.00","2,106.00"
36246939,malbertnews,"Albert, Mark",The Voyage Report,M,3575,28230,"1,078.00","1,151.00"


### All mentioned by beltway journalists
This is based on screen name, which could have changed during collection period. However, for the users that would be at the top of this list, seems unlikely.

In [33]:
# Mention count
mention_count_screen_name_df = pd.DataFrame(mention_df.mention_screen_name.value_counts().rename('mention_count'))

# Count of mentioning users
mention_user_id_per_user_screen_name_df = mention_df[['mention_screen_name', 'user_id']].drop_duplicates()
mentioning_count_screen_name_df = pd.DataFrame(mention_user_id_per_user_screen_name_df.groupby('mention_screen_name').size(), columns=['mentioning_count'])
mentioning_count_screen_name_df.index.name = 'screen_name'

all_mentioned_df = mention_count_screen_name_df.join(mentioning_count_screen_name_df)
all_mentioned_df.to_csv('output/all_mentioned_by_journalists.csv')
all_mentioned_df.head(25)

,mention_count,mentioning_count
realDonaldTrump,2876,452
POTUS,2265,253
wusa9,2111,41
AP,1948,143
USATODAY,1235,105
nbcwashington,1230,70
WSJ,1227,152
dcexaminer,1034,53
SHSanders45,927,148
nytimes,829,289


#### All mentioned by beltway journalists ordered by mentioning count

In [35]:
all_mentioned_df.sort_values(['mentioning_count', 'mention_count'], ascending=False).head(25)

,mention_count,mentioning_count
realDonaldTrump,2876,452
nytimes,829,289
POTUS,2265,253
SenJohnMcCain,599,231
Scaramucci,657,198
CNN,628,186
politico,747,181
SpeakerRyan,700,181
PressSec,654,178
washingtonpost,413,154


### All beltway journalists mentioned by beltway journalists

#### All beltway journalists mentioned by beltway journalists ordered by mention count

In [36]:
journalists_mention_summary_df = journalist_mention_summary(journalists_mention_df)
journalists_mention_summary_df.to_csv('output/journalists_mentioned_by_journalists.csv')
journalists_mention_summary_df[journalist_mention_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,mention_count,mentioning_count
user_id,,,,,,,
325050734,AllysonRaeWx,"Banks, Allyson",WUSA–TV,F,6918,330.00,7.00
28496589,TenaciousTopper,"Shutt, Charles",WUSA–TV,M,15868,239.00,13.00
63149389,hbwx,"Bernstein, Howard",WUSA–TV,M,8337,235.00,10.00
407013776,burgessev,"Everett, John B.",Politico,M,31010,212.00,46.00
16018516,jenhab,"Haberkorn, Jennifer A.",Politico,F,20028,200.00,31.00
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,143.00,41.00
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,127.00,51.00
169586280,WaPoSean,"Sullivan, Sean",Washington Post,M,22860,117.00,20.00
997684836,pkcapitol,"Kane, Paul",Washington Post,M,31300,116.00,47.00


#### All beltway journalists mentioned by beltway journalists ordered by mentioning count

In [37]:
journalists_mention_summary_df[journalist_mention_summary_fields].sort_values(['mentioning_count', 'mention_count'], ascending=False).head(25)

,screen_name,name,organization,gender,followers_count,mention_count,mentioning_count
user_id,,,,,,,
108617810,DanaBashCNN,"Bash, Dana",CNN,F,281861,115.00,55.00
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,127.00,51.00
997684836,pkcapitol,"Kane, Paul",Washington Post,M,31300,116.00,47.00
407013776,burgessev,"Everett, John B.",Politico,M,31010,212.00,46.00
112526560,kenvogel,"Vogel, Kenneth P.",Politico,M,53894,67.00,45.00
18227519,morningmika,"Brzezinski, Mika",MSNBC,F,653031,70.00,44.00
123327472,peterbakernyt,"Baker, Peter",New York Times,M,96956,107.00,43.00
39155029,mkraju,"Raju, Manu K.",CNN,M,88366,95.00,43.00
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,106.00,42.00


In [38]:
journalist_mention_gender_summary(journalists_mention_df)


,count,percentage
M,8298,58.0%
F,6000,42.0%


#### Stats of all beltway journalists mentioned by beltway journalists

In [39]:
journalists_mention_summary_df[['mention_count', 'mentioning_count']].describe()

,mention_count,mentioning_count
count,"2,292.00","2,292.00"
mean,6.24,2.94
std,17.59,5.65
min,0.00,0.00
25%,0.00,0.00
50%,1.00,1.00
75%,5.00,3.00
max,330.00,55.00


#### Top 100 journalists mentioned by beltway journalists

In [40]:
top_journalist_mention_gender_summary(journalists_mention_summary_df, mentioning_count_threshold=13, head=100)

,count,percentage
M,60,60.0%
F,40,40.0%


### Female beltway journalists mentioned by beltway journalists

In [41]:
female_journalists_mention_summary_df = journalists_mention_summary_df[journalists_mention_summary_df.gender == 'F']
female_journalists_mention_summary_df.to_csv('output/female_journalists_mentioned_by_journalists.csv')
female_journalists_mention_summary_df[journalist_mention_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,mention_count,mentioning_count
user_id,,,,,,,
325050734,AllysonRaeWx,"Banks, Allyson",WUSA–TV,F,6918,330.00,7.00
16018516,jenhab,"Haberkorn, Jennifer A.",Politico,F,20028,200.00,31.00
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,143.00,41.00
108617810,DanaBashCNN,"Bash, Dana",CNN,F,281861,115.00,55.00
82151660,kelsey_snell,"Snell, Kelse",Washington Post,F,8108,109.00,22.00
33919343,AshleyRParker,"Parker, Ashley",Washington Post,F,122382,100.00,31.00
52392666,ZoeTillman,"Tillman, Zoe",BuzzFeed,F,15246,87.00,14.00
26632935,HopeSeck,"Hodge Seck, Hope",Military.com,F,4584,83.00,3.00
16441088,jestei,"Steinhauer, Jennifer",New York Times,F,13452,76.00,26.00


#### Stats of female beltway journalists mentioned by beltway journalists

In [42]:
female_journalists_mention_summary_df[['mention_count', 'mentioning_count']].describe()

,mention_count,mentioning_count
count,993.00,993.00
mean,6.04,2.65
std,17.95,5.19
min,0.00,0.00
25%,0.00,0.00
50%,1.00,1.00
75%,4.00,3.00
max,330.00,55.00


### Male beltway journalists mentioned by beltway journalists

In [43]:
male_journalists_mention_summary_df = journalists_mention_summary_df[journalists_mention_summary_df.gender == 'M']
male_journalists_mention_summary_df.to_csv('output/male_journalists_mentioned_by_journalists.csv')
male_journalists_mention_summary_df[journalist_mention_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,mention_count,mentioning_count
user_id,,,,,,,
28496589,TenaciousTopper,"Shutt, Charles",WUSA–TV,M,15868,239.00,13.00
63149389,hbwx,"Bernstein, Howard",WUSA–TV,M,8337,235.00,10.00
407013776,burgessev,"Everett, John B.",Politico,M,31010,212.00,46.00
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,127.00,51.00
169586280,WaPoSean,"Sullivan, Sean",Washington Post,M,22860,117.00,20.00
997684836,pkcapitol,"Kane, Paul",Washington Post,M,31300,116.00,47.00
123327472,peterbakernyt,"Baker, Peter",New York Times,M,96956,107.00,43.00
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,106.00,42.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,105.00,27.00


#### Stats of male beltway journalists mentioned by beltway journalists

In [44]:
male_journalists_mention_summary_df[['mention_count', 'mentioning_count']].describe()

,mention_count,mentioning_count
count,"1,299.00","1,299.00"
mean,6.39,3.16
std,17.31,5.96
min,0.00,0.00
25%,0.00,0.00
50%,1.00,1.00
75%,5.00,4.00
max,239.00,51.00


### Beltway journalists mentioned by female beltway journalists

In [45]:
journalists_mentioned_by_female_summary_df = journalist_mention_summary(journalists_mention_df[journalists_mention_df.gender == 'F'])
journalists_mentioned_by_female_summary_df.to_csv('output/journalists_mentioned_by_female_journalists.csv')
journalists_mentioned_by_female_summary_df[journalist_mention_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,mention_count,mentioning_count
user_id,,,,,,,
407013776,burgessev,"Everett, John B.",Politico,M,31010,164.00,20.00
16018516,jenhab,"Haberkorn, Jennifer A.",Politico,F,20028,116.00,13.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,79.00,10.00
169586280,WaPoSean,"Sullivan, Sean",Washington Post,M,22860,71.00,11.00
48802204,HardballChris,"Matthews, Chris",NBC News,M,718330,70.00,3.00
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,64.00,16.00
22891564,chrisgeidner,"Geidner, Chris",BuzzFeed,M,83316,61.00,6.00
108617810,DanaBashCNN,"Bash, Dana",CNN,F,281861,60.00,26.00
16067683,pauldemko,"Demko, Paul Jeffrey",Politico,M,8170,57.00,10.00


In [46]:
journalist_mention_gender_summary(journalists_mention_df[journalists_mention_df.gender == 'F'])

,count,percentage
M,3162,54.8%
F,2605,45.2%


#### Top 100 journalists mentioned by female beltway journalists

In [47]:
top_journalist_mention_gender_summary(journalists_mentioned_by_female_summary_df)

,count,percentage
M,53,53.0%
F,47,47.0%


### Beltway journalists mentioned by male beltway journalists

In [48]:
journalists_mentioned_by_male_summary_df = journalist_mention_summary(journalists_mention_df[journalists_mention_df.gender == 'M'])
journalists_mentioned_by_male_summary_df.to_csv('output/journalists_mentioned_by_male_journalists.csv')
journalists_mentioned_by_male_summary_df[journalist_mention_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,mention_count,mentioning_count
user_id,,,,,,,
325050734,AllysonRaeWx,"Banks, Allyson",WUSA–TV,F,6918,324.00,4.00
28496589,TenaciousTopper,"Shutt, Charles",WUSA–TV,M,15868,225.00,7.00
63149389,hbwx,"Bernstein, Howard",WUSA–TV,M,8337,225.00,4.00
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,87.00,30.00
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,84.00,30.00
16018516,jenhab,"Haberkorn, Jennifer A.",Politico,F,20028,84.00,18.00
997684836,pkcapitol,"Kane, Paul",Washington Post,M,31300,81.00,34.00
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,79.00,25.00
123327472,peterbakernyt,"Baker, Peter",New York Times,M,96956,78.00,29.00


In [49]:
journalist_mention_gender_summary(journalists_mention_df[journalists_mention_df.gender == 'M'])

,count,percentage
M,5136,60.2%
F,3395,39.8%


#### Top 100 journalists mentioned by male beltway journalists

In [50]:
top_journalist_mention_gender_summary(journalists_mentioned_by_male_summary_df)

,count,percentage
M,59,59.0%
F,41,41.0%


## Retweet data prep

### Load retweets from tweets
Including retweets and quotes

In [51]:
# Simply the tweet on load
def retweet_transform(tweet):
    if tweet_type(tweet) in ('retweet', 'quote'):
        retweet = tweet.get('retweeted_status') or tweet.get('quoted_status')
        return {
            'tweet_id': tweet['id_str'],
            'user_id': tweet['user']['id_str'],
            'screen_name': tweet['user']['screen_name'],
            'retweet_user_id': retweet['user']['id_str'],
            'retweet_screen_name': retweet['user']['screen_name'],
            'tweet_created_at': date_parse(tweet['created_at'])            
        }
    return None

base_retweet_df = load_tweet_df(retweet_transform, ['tweet_id', 'user_id', 'screen_name', 'retweet_user_id',
                                           'retweet_screen_name', 'tweet_created_at'],
                           dedupe_columns=['tweet_id'])

base_retweet_df.count()

INFO:root:Loading from tweets/41feff28312c433ab004cd822212f4c2_001.json.gz
DEBUG:root:Loaded 50000
DEBUG:root:Loaded 100000
DEBUG:root:Loaded 150000
DEBUG:root:Loaded 200000
DEBUG:root:Loaded 250000
DEBUG:root:Loaded 300000
INFO:root:Loading from tweets/642bf140607547cb9d4c6b1fc49772aa_001.json.gz
DEBUG:root:Loaded 350000
DEBUG:root:Loaded 400000
DEBUG:root:Loaded 450000
DEBUG:root:Loaded 500000
DEBUG:root:Loaded 550000
INFO:root:Loading from tweets/9f7ed17c16a1494c8690b4053609539d_001.json.gz
DEBUG:root:Loaded 600000
DEBUG:root:Loaded 650000
DEBUG:root:Loaded 700000
DEBUG:root:Loaded 750000
DEBUG:root:Loaded 800000


tweet_id               456956
user_id                456956
screen_name            456956
retweet_user_id        456956
retweet_screen_name    456956
tweet_created_at       456956
dtype: int64

In [52]:
base_retweet_df.head()

,tweet_id,user_id,screen_name,retweet_user_id,retweet_screen_name,tweet_created_at
0,875012559228874752,15731368,HowardKurtz,5447242,emzanotti,2017-06-14 15:30:25+00:00
1,875010759759101952,15731368,HowardKurtz,18646108,BretBaier,2017-06-14 15:23:16+00:00
2,875010163496939520,15731368,HowardKurtz,1098615836,brookefoxnews,2017-06-14 15:20:54+00:00
3,875010062296784897,15731368,HowardKurtz,14173315,NBCNews,2017-06-14 15:20:30+00:00
4,874728109513334788,15731368,HowardKurtz,21619519,JoeNBC,2017-06-13 20:40:07+00:00


### Add gender of retweeter

In [53]:
retweet_df = base_retweet_df.join(user_summary_df['gender'], on='user_id')
retweet_df.count()

tweet_id               456956
user_id                456956
screen_name            456956
retweet_user_id        456956
retweet_screen_name    456956
tweet_created_at       456956
gender                 456956
dtype: int64

### Distinct retweeted users

In [54]:
retweet_df['retweet_user_id'].unique().size

49154

### Limit to retweeted journalists

In [55]:
journalists_retweet_df = retweet_df.join(user_summary_df['gender'], how='inner', on='retweet_user_id', rsuffix='_retweet')
journalists_retweet_df.rename(columns = {'gender_retweet': 'retweet_gender'}, inplace=True)
journalists_retweet_df.count()

tweet_id               117048
user_id                117048
screen_name            117048
retweet_user_id        117048
retweet_screen_name    117048
tweet_created_at       117048
gender                 117048
retweet_gender         117048
dtype: int64

In [56]:
journalists_retweet_df.head()

,tweet_id,user_id,screen_name,retweet_user_id,retweet_screen_name,tweet_created_at,gender,retweet_gender
1,875010759759101952,15731368,HowardKurtz,18646108,BretBaier,2017-06-14 15:23:16+00:00,M,M
1689,874962718457040896,552044121,cspanSusan,18646108,BretBaier,2017-06-14 12:12:22+00:00,F,M
2152,874964090028011520,117082566,TheBrodyFile,18646108,BretBaier,2017-06-14 12:17:49+00:00,M,M
2157,874702923468595204,117082566,TheBrodyFile,18646108,BretBaier,2017-06-13 19:00:02+00:00,M,M
9597,874739897839517697,70814224,jkeene1957,18646108,BretBaier,2017-06-13 21:26:58+00:00,M,M


### Functions for summarizing retweets by beltway journalists

In [57]:
# Gender of beltway journalists retweeted by beltway journalists
def journalist_retweet_gender_summary(retweet_df):
    return pd.DataFrame({'count':retweet_df.retweet_gender.value_counts(), 
                  'percentage': retweet_df.retweet_gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

def journalist_retweet_summary(retweet_df):
    # Retweet count
    retweet_count_df = pd.DataFrame(retweet_df.retweet_user_id.value_counts().rename('retweet_count'))

    # Retweeting users. That is, the number of unique users retweeting each user.
    retweet_user_id_per_user_df = retweet_df[['retweet_user_id', 'user_id']].drop_duplicates()
    retweeting_user_count_df = pd.DataFrame(retweet_user_id_per_user_df.groupby('retweet_user_id').size(), columns=['retweeting_count'])
    retweeting_user_count_df.index.name = 'user_id'

    # Join with user summary
    journalist_retweet_summary_df = user_summary_df.join([retweet_count_df, retweeting_user_count_df])
    journalist_retweet_summary_df.fillna(0, inplace=True)
    journalist_retweet_summary_df = journalist_retweet_summary_df.sort_values(['retweet_count', 'retweeting_count', 'followers_count'], ascending=False)
    return journalist_retweet_summary_df

# Gender of top journalists retweeted by beltway journalists
def top_journalist_retweet_gender_summary(retweet_summary_df, retweeting_count_threshold=0, head=100):
    top_retweet_summary_df = retweet_summary_df[retweet_summary_df.retweeting_count > retweeting_count_threshold].head(head)
    return pd.DataFrame({'count': top_retweet_summary_df.gender.value_counts(), 
                  'percentage': top_retweet_summary_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

# Fields for displaying journalist mention summaries
journalist_retweet_summary_fields = ['screen_name', 'name', 'organization', 'gender', 'followers_count', 'retweet_count', 'retweeting_count']


## Retweet analysis
*Note that for each of these, the complete list is being written to CSV in the output directory.*


### Retweets and quotes by gender

In [58]:
retweets_by_gender_df = user_summary_df[['gender', 'retweet', 'quote']].groupby('gender').sum()
retweets_by_gender_df['total'] = retweets_by_gender_df.retweet + retweets_by_gender_df.quote
retweets_by_gender_df['percentage'] = retweets_by_gender_df.total.div(retweets_by_gender_df.total.sum()).mul(100).round(1).astype(str) + '%'
retweets_by_gender_df

,retweet,quote,total,percentage
gender,,,,
F,"134,606.00","38,998.00","173,604.00",38.0%
M,"210,660.00","72,692.00","283,352.00",62.0%


### Most prolific retweeters

In [59]:
retweet_user_summary_df = user_summary_df.loc[:,('screen_name', 'name', 'organization', 'gender', 'followers_count', 'tweet_count', 'retweet', 'quote', 'tweets_in_dataset')]
retweet_user_summary_df['retweet_count'] = retweet_user_summary_df.retweet + retweet_user_summary_df.quote
retweet_user_summary_df.sort_values(['retweet_count'], ascending=False).head(25)

,screen_name,name,organization,gender,followers_count,tweet_count,retweet,quote,tweets_in_dataset,retweet_count
user_id,,,,,,,,,,
2453025128,gloriaminott,"Minott, Gloria",WPFW–FM,F,586,61473,"21,524.00",0.00,"21,547.00","21,524.00"
304988603,NeilWMcCabe,"McCabe, Neil",Breitbart News,M,18903,64673,"7,528.00",625.00,"9,370.00","8,153.00"
18825339,CahnEmily,"Cahn, Emily",Mic,F,16980,100803,"4,449.00","1,834.00","8,196.00","6,283.00"
191964162,SamLitzinger,"Litzinger, Sam",CBS News,M,2329,95236,"6,017.00",225.00,"7,537.00","6,242.00"
21612122,HotlineJosh,"Kraushaar, Josh P.",National Journal,M,50438,156610,"4,881.00",893.00,"6,703.00","5,774.00"
259395895,JohnJHarwood,"Harwood, John",CNBC,M,149040,78015,"4,570.00",822.00,"6,377.00","5,392.00"
16031927,greta,"Van Susteren, Greta",MSNBC,F,1186850,116645,794.00,"3,069.00","4,792.00","3,863.00"
21810329,sdonnan,"Donnan, Shawn",Financial Times,M,12311,79125,"3,332.00",449.00,"4,537.00","3,781.00"
47408060,JonathanLanday,"Landay, Jonathan",McClatchy Newspapers,M,11213,81042,"3,687.00",80.00,"4,285.00","3,767.00"


### All retweeted by beltway journalists
This is based on screen name, which could have changed during collection period. However, for the users that would be at the top of this list, seems unlikely.

In [60]:
# Retweet count
retweet_count_screen_name_df = pd.DataFrame(retweet_df.retweet_screen_name.value_counts().rename('retweet_count'))

# Count of retweeting users
retweet_user_id_per_user_screen_name_df = retweet_df[['retweet_screen_name', 'user_id']].drop_duplicates()
retweeting_count_screen_name_df = pd.DataFrame(retweet_user_id_per_user_screen_name_df.groupby('retweet_screen_name').size(), columns=['retweeting_count'])
retweeting_count_screen_name_df.index.name = 'screen_name'

all_retweeted_df = retweet_count_screen_name_df.join(retweeting_count_screen_name_df)
all_retweeted_df.to_csv('output/all_retweeted_by_journalists.csv')
all_retweeted_df.head(25)

,retweet_count,retweeting_count
realDonaldTrump,6650,807
thehill,5424,457
BraddJaffy,3564,554
maggieNYT,3024,530
business,3000,229
washingtonpost,2638,498
AP,2480,581
politico,2335,334
nytimes,2268,485
WSJ,1949,213


### All beltway journalists retweeted by beltway journalists

In [61]:
journalists_retweet_summary_df = journalist_retweet_summary(journalists_retweet_df)
journalists_retweet_summary_df.to_csv('output/journalists_retweeted_by_journalists.csv')
journalists_retweet_summary_df[journalist_retweet_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,retweet_count,retweeting_count
user_id,,,,,,,
407013776,burgessev,"Everett, John B.",Politico,M,31010,"1,836.00",289.00
21316253,ZekeJMiller,"Miller, Zeke J.",Time Magazine,M,198517,"1,723.00",387.00
19107878,GlennThrush,"Thrush, Glenn H.",New York Times,M,308181,"1,577.00",451.00
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,"1,459.00",397.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,"1,403.00",280.00
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,"1,393.00",327.00
39155029,mkraju,"Raju, Manu K.",CNN,M,88366,"1,359.00",341.00
31127446,markknoller,"Knoller, Mark",CBS News,M,301474,"1,343.00",341.00
398088661,MEPFuller,"Fuller, Matt E.",Huffington Post,M,77919,"1,324.00",286.00


In [62]:
journalist_retweet_gender_summary(journalists_retweet_df)


,count,percentage
M,80634,68.9%
F,36414,31.1%


#### Stats of beltway journalists retweeted by beltway journalists

In [63]:
journalists_retweet_summary_df[['retweet_count', 'retweeting_count']].describe()

,retweet_count,retweeting_count
count,"2,292.00","2,292.00"
mean,51.07,19.84
std,149.06,45.38
min,0.00,0.00
25%,0.00,0.00
50%,6.00,4.00
75%,33.00,16.00
max,"1,836.00",451.00


#### Top 100 journalists retweeted by beltway journalists

In [64]:
top_journalist_retweet_gender_summary(journalists_retweet_summary_df)

,count,percentage
M,78,78.0%
F,22,22.0%


### Female beltway journalists retweeted by beltway journalists

In [65]:
female_journalists_retweet_summary_df = journalists_retweet_summary_df[journalists_retweet_summary_df.gender == 'F']
female_journalists_retweet_summary_df.to_csv('output/female_journalists_retweeted_by_journalists.csv')
female_journalists_retweet_summary_df[journalist_retweet_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,retweet_count,retweeting_count
user_id,,,,,,,
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,"1,393.00",327.00
33653195,ericawerner,"Werner, Erica",Associated Press,F,14049,939.00,281.00
12354832,kasie,"Hunt, Kasie",NBC News,F,187357,909.00,388.00
70511174,Hadas_Gold,"Gold, Hadas",Politico,F,45221,849.00,306.00
593813785,DonnaYoungDC,"Young, Donna",S&P Global Market Intelligence,F,5894,708.00,13.00
167024520,rachaelmbade,"Bade, Rachel M.",Politico,F,30164,614.00,161.00
33919343,AshleyRParker,"Parker, Ashley",Washington Post,F,122382,539.00,268.00
139738464,mj_lee,"Lee, MJ",CNN,F,31940,518.00,189.00
16018516,jenhab,"Haberkorn, Jennifer A.",Politico,F,20028,474.00,136.00


#### Stats of female beltway journalists retweeted by beltway journalists

In [66]:
female_journalists_retweet_summary_df[['retweet_count', 'retweeting_count']].describe()

,retweet_count,retweeting_count
count,993.00,993.00
mean,36.67,15.33
std,97.34,34.51
min,0.00,0.00
25%,0.00,0.00
50%,5.00,3.00
75%,25.00,13.00
max,"1,393.00",388.00


### Male beltway journalists retweeted by beltway journalists

In [67]:
male_journalists_retweet_summary_df = journalists_retweet_summary_df[journalists_retweet_summary_df.gender == 'M']
male_journalists_retweet_summary_df.to_csv('output/male_journalists_retweeted_by_journalists.csv')
male_journalists_retweet_summary_df[journalist_retweet_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,retweet_count,retweeting_count
user_id,,,,,,,
407013776,burgessev,"Everett, John B.",Politico,M,31010,"1,836.00",289.00
21316253,ZekeJMiller,"Miller, Zeke J.",Time Magazine,M,198517,"1,723.00",387.00
19107878,GlennThrush,"Thrush, Glenn H.",New York Times,M,308181,"1,577.00",451.00
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,"1,459.00",397.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,"1,403.00",280.00
39155029,mkraju,"Raju, Manu K.",CNN,M,88366,"1,359.00",341.00
31127446,markknoller,"Knoller, Mark",CBS News,M,301474,"1,343.00",341.00
398088661,MEPFuller,"Fuller, Matt E.",Huffington Post,M,77919,"1,324.00",286.00
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,"1,221.00",306.00


#### Stats of male beltway journalists retweeted by beltway journalists

In [68]:
male_journalists_retweet_summary_df[['retweet_count', 'retweeting_count']].describe()

,retweet_count,retweeting_count
count,"1,299.00","1,299.00"
mean,62.07,23.28
std,178.04,51.94
min,0.00,0.00
25%,1.00,1.00
50%,8.00,5.00
75%,39.50,19.00
max,"1,836.00",451.00


### Beltway journalists retweeted by female beltway journalists

In [69]:
journalists_retweeted_by_female_summary_df = journalist_retweet_summary(journalists_retweet_df[journalists_retweet_df.gender == 'F'])
journalists_retweeted_by_female_summary_df.to_csv('output/journalists_retweeted_by_female_journalists.csv')
journalists_retweeted_by_female_summary_df[journalist_retweet_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,retweet_count,retweeting_count
user_id,,,,,,,
407013776,burgessev,"Everett, John B.",Politico,M,31010,748.00,122.00
593813785,DonnaYoungDC,"Young, Donna",S&P Global Market Intelligence,F,5894,704.00,9.00
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,572.00,142.00
31127446,markknoller,"Knoller, Mark",CBS News,M,301474,549.00,140.00
21316253,ZekeJMiller,"Miller, Zeke J.",Time Magazine,M,198517,516.00,149.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,503.00,97.00
14007532,frankthorp,"Thorp, Frank",NBC News,M,39798,470.00,140.00
19107878,GlennThrush,"Thrush, Glenn H.",New York Times,M,308181,463.00,165.00
33653195,ericawerner,"Werner, Erica",Associated Press,F,14049,452.00,119.00


In [70]:
journalist_retweet_gender_summary(journalists_retweet_df[journalists_retweet_df.gender == 'F'])


,count,percentage
M,25410,59.6%
F,17228,40.4%


#### Top 100 journalists retweeted by female beltway journalists

In [71]:
top_journalist_retweet_gender_summary(journalists_retweeted_by_female_summary_df)

,count,percentage
M,59,59.0%
F,41,41.0%


### Beltway journalists retweeted by male beltway journalists

In [72]:
journalists_retweeted_by_male_summary_df = journalist_retweet_summary(journalists_retweet_df[journalists_retweet_df.gender == 'M'])
journalists_retweeted_by_male_summary_df.to_csv('output/journalists_retweeted_by_male_journalists.csv')
journalists_retweeted_by_male_summary_df[journalist_retweet_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,retweet_count,retweeting_count
user_id,,,,,,,
21316253,ZekeJMiller,"Miller, Zeke J.",Time Magazine,M,198517,"1,207.00",238.00
19107878,GlennThrush,"Thrush, Glenn H.",New York Times,M,308181,"1,114.00",286.00
407013776,burgessev,"Everett, John B.",Politico,M,31010,"1,088.00",167.00
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,"1,071.00",239.00
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,975.00,209.00
39155029,mkraju,"Raju, Manu K.",CNN,M,88366,956.00,209.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,900.00,183.00
398088661,MEPFuller,"Fuller, Matt E.",Huffington Post,M,77919,877.00,170.00
19847765,sahilkapur,"Kapur, Sahil",Bloomberg News,M,69086,848.00,193.00


In [73]:
journalist_retweet_gender_summary(journalists_retweet_df[journalists_retweet_df.gender == 'M'])

,count,percentage
M,55224,74.2%
F,19186,25.8%


#### Top 100 journalists retweeted by male beltway journalists

In [74]:
top_journalist_retweet_gender_summary(journalists_retweeted_by_male_summary_df)

,count,percentage
M,80,80.0%
F,20,20.0%


## Reply data prep

### Load replies from tweets

In [75]:
# Simply the tweet on load
def reply_transform(tweet):
    if tweet_type(tweet) == 'reply':        
        return {
            'tweet_id': tweet['id_str'],
            'user_id': tweet['user']['id_str'],
            'screen_name': tweet['user']['screen_name'],
            'reply_to_user_id': tweet['in_reply_to_user_id_str'],
            'reply_to_screen_name': tweet['in_reply_to_screen_name'],
            'tweet_created_at': date_parse(tweet['created_at'])            
        }
    return None

base_reply_df = load_tweet_df(reply_transform, ['tweet_id', 'user_id', 'screen_name', 'reply_to_user_id',
                                           'reply_to_screen_name', 'tweet_created_at'],
                           dedupe_columns=['tweet_id'])

base_reply_df.count()

INFO:root:Loading from tweets/41feff28312c433ab004cd822212f4c2_001.json.gz
DEBUG:root:Loaded 50000
DEBUG:root:Loaded 100000
DEBUG:root:Loaded 150000
DEBUG:root:Loaded 200000
DEBUG:root:Loaded 250000
DEBUG:root:Loaded 300000
INFO:root:Loading from tweets/642bf140607547cb9d4c6b1fc49772aa_001.json.gz
DEBUG:root:Loaded 350000
DEBUG:root:Loaded 400000
DEBUG:root:Loaded 450000
DEBUG:root:Loaded 500000
DEBUG:root:Loaded 550000
INFO:root:Loading from tweets/9f7ed17c16a1494c8690b4053609539d_001.json.gz
DEBUG:root:Loaded 600000
DEBUG:root:Loaded 650000
DEBUG:root:Loaded 700000
DEBUG:root:Loaded 750000
DEBUG:root:Loaded 800000


tweet_id                126254
user_id                 126254
screen_name             126254
reply_to_user_id        126254
reply_to_screen_name    126254
tweet_created_at        126254
dtype: int64

In [76]:
base_reply_df.head()

,tweet_id,user_id,screen_name,reply_to_user_id,reply_to_screen_name,tweet_created_at
0,873663547670237184,15731368,HowardKurtz,823693635564802048,SHSanders45,2017-06-10 22:09:56+00:00
1,873590575999700992,15731368,HowardKurtz,2263438704,GillianHTurner,2017-06-10 17:19:58+00:00
2,874830107034210304,620618811,robyoon,79743108,alexburnsNYT,2017-06-14 03:25:25+00:00
3,875111766744870913,2421093038,SutherlandFox,593995023,dre_102,2017-06-14 22:04:38+00:00
4,875104971255156736,2421093038,SutherlandFox,593995023,dre_102,2017-06-14 21:37:38+00:00


### Add gender of replier

In [77]:
reply_df = base_reply_df.join(user_summary_df['gender'], on='user_id')
reply_df.count()

tweet_id                126254
user_id                 126254
screen_name             126254
reply_to_user_id        126254
reply_to_screen_name    126254
tweet_created_at        126254
gender                  126254
dtype: int64

### Distinct replied to users

In [78]:
reply_df['reply_to_user_id'].unique().size

31034

### Limit to beltway journalists

In [79]:
journalists_reply_df = reply_df.join(user_summary_df['gender'], how='inner', on='reply_to_user_id', rsuffix='_reply')
journalists_reply_df.rename(columns = {'gender_reply': 'reply_to_gender'}, inplace=True)
journalists_reply_df.count()

tweet_id                43390
user_id                 43390
screen_name             43390
reply_to_user_id        43390
reply_to_screen_name    43390
tweet_created_at        43390
gender                  43390
reply_to_gender         43390
dtype: int64

In [80]:
journalists_reply_df.head()

,tweet_id,user_id,screen_name,reply_to_user_id,reply_to_screen_name,tweet_created_at,gender,reply_to_gender
7,875145443231092736,26607712,MattLaslo,26607712,MattLaslo,2017-06-15 00:18:27+00:00,M,M
8,875145257767362560,26607712,MattLaslo,26607712,MattLaslo,2017-06-15 00:17:43+00:00,M,M
19,872900196971278336,26607712,MattLaslo,26607712,MattLaslo,2017-06-08 19:36:39+00:00,M,M
1593,874712583189798912,17874756,AnthonyJRivera,26607712,MattLaslo,2017-06-13 19:38:25+00:00,M,M
8697,872610739885920256,1246438250,jimhoward529,26607712,MattLaslo,2017-06-08 00:26:27+00:00,M,M


### Functions for summarizing replies by beltway journalists

In [81]:
# Gender of beltway journalists replied to by beltway journalists
def journalist_reply_gender_summary(reply_df):
    return pd.DataFrame({'count':reply_df.reply_to_gender.value_counts(), 
                  'percentage': reply_df.reply_to_gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

# Reply to beltway journalists by beltway journalists
def journalist_reply_summary(reply_df):
    # Reply to count
    reply_count_df = pd.DataFrame(reply_df.reply_to_user_id.value_counts().rename('reply_to_count'))
    
    # Replying to users. That is, the number of unique users replying to each user.
    reply_to_user_id_per_user_df = reply_df[['reply_to_user_id', 'user_id']].drop_duplicates()
    replying_to_user_count_df = pd.DataFrame(reply_to_user_id_per_user_df.groupby('reply_to_user_id').size(), columns=['replying_count'])
    replying_to_user_count_df.index.name = 'user_id'
    
    # Join with user summary
    journalist_reply_summary_df = user_summary_df.join([reply_count_df, replying_to_user_count_df])
    journalist_reply_summary_df.fillna(0, inplace=True)
    journalist_reply_summary_df = journalist_reply_summary_df.sort_values(['reply_to_count', 'replying_count', 'followers_count'], ascending=False)
    return journalist_reply_summary_df

# Gender of top journalists replied to by beltway journalists
def top_journalist_reply_gender_summary(reply_summary_df, replying_count_threshold=0, head=100):
    top_reply_summary_df = reply_summary_df[reply_summary_df.replying_count > replying_count_threshold].head(head)
    return pd.DataFrame({'count': top_reply_summary_df.gender.value_counts(), 
                  'percentage': top_reply_summary_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

# Fields for displaying journalist mention summaries
journalist_reply_summary_fields = ['screen_name', 'name', 'organization', 'gender', 'followers_count', 'reply_to_count', 'replying_count']


## Reply analysis
*Note that for each of these, the complete list is being written to CSV in the output directory.*


### Replies by gender

In [82]:
replies_by_gender_df = user_summary_df[['gender', 'reply']].groupby('gender').sum()
replies_by_gender_df['percentage'] = replies_by_gender_df.reply.div(replies_by_gender_df.reply.sum()).mul(100).round(1).astype(str) + '%'
replies_by_gender_df

,reply,percentage
gender,,
F,"31,831.00",25.2%
M,"94,423.00",74.8%


### Most prolific repliers

In [83]:
user_summary_df[['screen_name', 'name', 'organization', 'gender', 'followers_count', 'tweet_count', 'reply', 'tweets_in_dataset']].sort_values(['reply'], ascending=False).head(25)

,screen_name,name,organization,gender,followers_count,tweet_count,reply,tweets_in_dataset
user_id,,,,,,,,
3817401,ericgeller,"Geller, Eric",Politico,M,58173,208763,"9,033.00","11,432.00"
22891564,chrisgeidner,"Geidner, Chris",BuzzFeed,M,83316,205504,"3,917.00","6,244.00"
118130765,dylanlscott,"Scott, Dylan L.",Stat News,M,20122,42497,"2,040.00","3,960.00"
19576571,JaredRizzi,"Rizzi, Jared",Sirius XM Satellite Radio,M,13545,41620,"1,949.00","5,567.00"
275207082,AlexParkerDC,"Parker, Alexander M.",Bloomberg BNA,M,3828,142150,"1,714.00","3,983.00"
63717541,phillyrich1,"Weinstein, Richard",C–SPAN,M,3827,27341,"1,532.00","2,261.00"
203226736,SharylAttkisson,"Attkisson, Sharyl",Sinclair Broadcast Group,F,132973,24539,"1,458.00","2,154.00"
16812908,crousselle,"Rousselle, Christine",Townhall,F,5327,118713,"1,089.00","2,351.00"
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,148143,"1,040.00","5,078.00"


### All replied to by beltway journalists
This is based on screen name, which could have changed during collection period. However, for the users that would be at the top of this list, seems unlikely.

In [84]:
# Reply to count
reply_to_count_screen_name_df = pd.DataFrame(reply_df.reply_to_screen_name.value_counts().rename('reply_to_count'))

# Count of replying users
reply_to_user_id_per_user_screen_name_df = reply_df[['reply_to_screen_name', 'user_id']].drop_duplicates()
replying_count_screen_name_df = pd.DataFrame(reply_to_user_id_per_user_screen_name_df.groupby('reply_to_screen_name').size(), columns=['replying_count'])
replying_count_screen_name_df.index.name = 'screen_name'

all_replied_to_df = reply_to_count_screen_name_df.join(replying_count_screen_name_df)
all_replied_to_df.to_csv('output/all_replied_to_by_journalists.csv')
all_replied_to_df.head(25)

,reply_to_count,replying_count
ericgeller,1980,75
chrisgeidner,1901,37
dylanlscott,1091,65
JaredRizzi,750,46
StevenTDennis,745,93
AlexParkerDC,720,23
sahilkapur,662,35
jseldin,653,2
MEPFuller,522,92
amaxsmith,498,6


### All beltway journalists replied to by beltway journalists

In [85]:
journalists_reply_summary_df = journalist_reply_summary(journalists_reply_df)
journalists_reply_summary_df.to_csv('output/journalists_replied_to_by_journalists.csv')
journalists_reply_summary_df[journalist_reply_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,reply_to_count,replying_count
user_id,,,,,,,
3817401,ericgeller,"Geller, Eric",Politico,M,58173,"1,980.00",75.00
22891564,chrisgeidner,"Geidner, Chris",BuzzFeed,M,83316,"1,901.00",37.00
118130765,dylanlscott,"Scott, Dylan L.",Stat News,M,20122,"1,091.00",65.00
19576571,JaredRizzi,"Rizzi, Jared",Sirius XM Satellite Radio,M,13545,750.00,46.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,745.00,93.00
275207082,AlexParkerDC,"Parker, Alexander M.",Bloomberg BNA,M,3828,720.00,23.00
19847765,sahilkapur,"Kapur, Sahil",Bloomberg News,M,69086,662.00,35.00
583821006,jseldin,"Seldin, Jeff",Voice of America,M,5365,653.00,2.00
398088661,MEPFuller,"Fuller, Matt E.",Huffington Post,M,77919,522.00,92.00


In [86]:
journalist_reply_gender_summary(journalists_reply_df)

,count,percentage
M,33178,76.5%
F,10212,23.5%


#### Stats of beltway journalists replied to by beltway journalists

In [87]:
journalists_reply_summary_df[['reply_to_count', 'replying_count']].describe()

,reply_to_count,replying_count
count,"2,292.00","2,292.00"
mean,18.93,3.81
std,81.76,8.41
min,0.00,0.00
25%,0.00,0.00
50%,1.00,1.00
75%,8.00,4.00
max,"1,980.00",93.00


#### Top 100 journalists replied to by beltway journalists

In [88]:
top_journalist_reply_gender_summary(journalists_reply_summary_df, replying_count_threshold=0)

,count,percentage
M,81,81.0%
F,19,19.0%


# Female beltway journalists replied to by beltway journalists

In [89]:
female_journalists_reply_summary_df = journalists_reply_summary_df[journalists_reply_summary_df.gender == 'F']
female_journalists_reply_summary_df.to_csv('output/female_journalists_replied_to_by_journalists.csv')
female_journalists_reply_summary_df[journalist_reply_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,reply_to_count,replying_count
user_id,,,,,,,
16061946,kelmej,"Mejdrich, Kellie",CQ Roll Call,F,4146,340.00,29.00
906734342,KimberlyRobinsn,"Robinson, Kimberly S.",Bloomberg BNA,F,7170,321.00,7.00
52392666,ZoeTillman,"Tillman, Zoe",BuzzFeed,F,15246,311.00,8.00
21212087,Olivianuzzi,"Nuzzi, Olivia",New York,F,136276,243.00,25.00
83462293,SarahMMimms,"Mimms, Sarah",BuzzFeed,F,6216,236.00,24.00
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,233.00,84.00
3372900155,samtayrey,"Reyes, Samantha",CNN,F,10344,219.00,18.00
18825339,CahnEmily,"Cahn, Emily",Mic,F,16980,212.00,48.00
1132012321,DaniellaMicaela,"Diaz, Daniella",CNN,F,14612,181.00,36.00


#### Stats of female beltway journalists replied to by beltway journalists

In [90]:
female_journalists_reply_summary_df[['reply_to_count', 'replying_count']].describe()

,reply_to_count,replying_count
count,993.00,993.00
mean,10.28,2.95
std,31.00,6.33
min,0.00,0.00
25%,0.00,0.00
50%,1.00,1.00
75%,6.00,3.00
max,340.00,84.00


### Male beltway journalists replied to by beltway journalists

In [91]:
male_journalists_reply_summary_df = journalists_reply_summary_df[journalists_reply_summary_df.gender == 'M']
male_journalists_reply_summary_df.to_csv('output/male_journalists_replied_to_by_journalists.csv')
male_journalists_reply_summary_df[journalist_reply_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,reply_to_count,replying_count
user_id,,,,,,,
3817401,ericgeller,"Geller, Eric",Politico,M,58173,"1,980.00",75.00
22891564,chrisgeidner,"Geidner, Chris",BuzzFeed,M,83316,"1,901.00",37.00
118130765,dylanlscott,"Scott, Dylan L.",Stat News,M,20122,"1,091.00",65.00
19576571,JaredRizzi,"Rizzi, Jared",Sirius XM Satellite Radio,M,13545,750.00,46.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,745.00,93.00
275207082,AlexParkerDC,"Parker, Alexander M.",Bloomberg BNA,M,3828,720.00,23.00
19847765,sahilkapur,"Kapur, Sahil",Bloomberg News,M,69086,662.00,35.00
583821006,jseldin,"Seldin, Jeff",Voice of America,M,5365,653.00,2.00
398088661,MEPFuller,"Fuller, Matt E.",Huffington Post,M,77919,522.00,92.00


#### Stats of male beltway journalists replied to by beltway journalists

In [92]:
male_journalists_reply_summary_df[['reply_to_count', 'replying_count']].describe()

,reply_to_count,replying_count
count,"1,299.00","1,299.00"
mean,25.54,4.46
std,104.71,9.66
min,0.00,0.00
25%,0.00,0.00
50%,1.00,1.00
75%,11.00,4.00
max,"1,980.00",93.00


### Beltway journalists replied to by female beltway journalists

In [93]:
journalists_replied_to_by_female_summary_df = journalist_reply_summary(journalists_reply_df[journalists_reply_df.gender == 'F'])
journalists_replied_to_by_female_summary_df.to_csv('output/journalists_replied_to_by_female_journalists.csv')
journalists_replied_to_by_female_summary_df[journalist_reply_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,reply_to_count,replying_count
user_id,,,,,,,
906734342,KimberlyRobinsn,"Robinson, Kimberly S.",Bloomberg BNA,F,7170,313.00,2.00
52392666,ZoeTillman,"Tillman, Zoe",BuzzFeed,F,15246,305.00,3.00
16061946,kelmej,"Mejdrich, Kellie",CQ Roll Call,F,4146,295.00,15.00
83462293,SarahMMimms,"Mimms, Sarah",BuzzFeed,F,6216,195.00,7.00
21212087,Olivianuzzi,"Nuzzi, Olivia",New York,F,136276,190.00,9.00
3372900155,samtayrey,"Reyes, Samantha",CNN,F,10344,179.00,7.00
96405362,laurenonthehill,"Camera, Lauren S.",U.S. News & World Report,F,3396,159.00,5.00
18825339,CahnEmily,"Cahn, Emily",Mic,F,16980,148.00,18.00
1132012321,DaniellaMicaela,"Diaz, Daniella",CNN,F,14612,144.00,22.00


In [94]:
journalist_reply_gender_summary(journalists_reply_df[journalists_reply_df.gender == 'F'])


,count,percentage
F,7412,72.1%
M,2864,27.9%


#### Top 100 journalists replied to by female beltway journalists

In [95]:
top_journalist_reply_gender_summary(journalists_replied_to_by_female_summary_df, replying_count_threshold=0)

,count,percentage
F,75,75.0%
M,25,25.0%


### Beltway journalists replied to by male beltway journalists

In [96]:
journalists_replied_to_by_male_summary_df = journalist_reply_summary(journalists_reply_df[journalists_reply_df.gender == 'M'])
journalists_replied_to_by_male_summary_df.to_csv('output/journalists_replied_to_by_male_journalists.csv')
journalists_replied_to_by_male_summary_df[journalist_reply_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,reply_to_count,replying_count
user_id,,,,,,,
3817401,ericgeller,"Geller, Eric",Politico,M,58173,"1,926.00",58.00
22891564,chrisgeidner,"Geidner, Chris",BuzzFeed,M,83316,"1,864.00",28.00
118130765,dylanlscott,"Scott, Dylan L.",Stat News,M,20122,"1,013.00",45.00
19576571,JaredRizzi,"Rizzi, Jared",Sirius XM Satellite Radio,M,13545,726.00,35.00
275207082,AlexParkerDC,"Parker, Alexander M.",Bloomberg BNA,M,3828,709.00,20.00
46557945,StevenTDennis,"Dennis, Steven T.",Bloomberg News,M,55762,686.00,61.00
583821006,jseldin,"Seldin, Jeff",Voice of America,M,5365,653.00,2.00
19847765,sahilkapur,"Kapur, Sahil",Bloomberg News,M,69086,646.00,24.00
44951698,amaxsmith,"Smith, Max",WTOP Radio,M,4726,495.00,4.00


In [97]:
journalists_replied_to_by_male_summary_df.count()

screen_name          2292
name                 2292
organization         2292
position             2292
gender               2292
followers_count      2292
following_count      2292
tweet_count          2292
user_created_at      2292
verified             2292
protected            2292
original             2292
quote                2292
reply                2292
retweet              2292
tweets_in_dataset    2292
reply_to_count       2292
replying_count       2292
dtype: int64

In [98]:
journalist_reply_gender_summary(journalists_reply_df[journalists_reply_df.gender == 'M'])

,count,percentage
M,30314,91.5%
F,2800,8.5%


#### Top 100 journalists replied to by male beltway journalists

In [99]:
top_journalist_reply_gender_summary(journalists_replied_to_by_male_summary_df, replying_count_threshold=0)

,count,percentage
M,98,98.0%
F,2,2.0%


## Following data prep

### Load following
Users that are followed by beltway journalists

In [100]:
base_follower_to_followed_df = pd.read_csv('source_data/follower_to_followed.csv', 
                                      names=['follower_user_id', 'followed_user_id'],
                                      dtype={'follower_user_id': np.str, 'followed_user_id': np.str})
base_follower_to_followed_df.drop_duplicates(inplace=True)
base_follower_to_followed_df.count()

follower_user_id    3417018
followed_user_id    3417018
dtype: int64

In [101]:
base_follower_to_followed_df.head()

,follower_user_id,followed_user_id
0,91156486,3092427779
1,91156486,36953109
2,91156486,424274008
3,91156486,779044378929168384
4,91156486,339834914


In [102]:
user_info_df.head()

,name,organization,position,gender,followers_count,following_count,tweet_count,user_created_at,verified,protected
user_id,,,,,,,,,,
20711445,"Glinski, Nina",NaN,Freelance Reporter,F,963,507,909,Thu Feb 12 20:00:53 +0000 2009,False,False
258917371,"Enders, David",NaN,Journalist,M,1444,484,6296,Mon Feb 28 19:52:03 +0000 2011,True,False
297046834,"Barakat, Matthew",Associated Press,Northern Virginia Correspondent,M,759,352,631,Wed May 11 20:55:24 +0000 2011,True,False
455585786,"Atkins, Kimberly",Boston Herald,Chief Washington Reporter/Columnist,F,2944,2691,6277,Thu Jan 05 08:26:46 +0000 2012,True,False
42584840,"Vlahou, Toula",CQ Roll Call,Editor & Podcast Producer,F,2703,201,6366,Tue May 26 07:41:38 +0000 2009,False,False


In [103]:
# This will drop followers of journalists that have no tweets
follower_to_followed_df = base_follower_to_followed_df.join(user_summary_df['gender'], on='follower_user_id', how='inner')
follower_to_followed_df.count()

follower_user_id    3311406
followed_user_id    3311406
gender              3311406
dtype: int64

In [104]:
follower_to_followed_df.head()

,follower_user_id,followed_user_id,gender
261,15219888,3291076716,F
262,15219888,119175339,F
263,15219888,418837047,F
264,15219888,259817885,F
265,15219888,287263845,F


### Load followed users

In [105]:
followed_screen_name_lookup_df = pd.read_csv('source_data/followed.csv', 
                                             names=['screen_name', 'user_id'],
                                             dtype={'user_id': np.str}).set_index(['user_id'])
followed_screen_name_lookup_df.head()

,screen_name
user_id,
17665874,onlinehigh
2389275799,HLSPOLICY
314728983,Veolia_NA
239409802,fishingbuk
522799320,GoldsmithBev


### Limit to beltway journalists

In [106]:
follower_to_journalist_followed_df = follower_to_followed_df.join(user_summary_df['gender'], how='inner', on='followed_user_id', rsuffix='_followed')
follower_to_journalist_followed_df.rename(columns = {'gender_followed': 'followed_gender'}, inplace=True)
follower_to_journalist_followed_df.count()

follower_user_id    280340
followed_user_id    280340
gender              280340
followed_gender     280340
dtype: int64

In [107]:
follower_to_journalist_followed_df.head()

,follower_user_id,followed_user_id,gender,followed_gender
287,15219888,46582653,F,M
21810,15780280,46582653,M,M
24153,14245722,46582653,M,M
40694,37865281,46582653,F,M
66585,165204211,46582653,M,M


### Functions for summarizing following by beltway journalists

In [108]:
# Gender of beltway journalists followed by beltway journalists
def journalist_followed_gender_summary(follower_to_followed_df):
    return pd.DataFrame({'count':follower_to_followed_df.followed_gender.value_counts(), 
                  'percentage': follower_to_followed_df.followed_gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

# Following of beltway journalists by beltway journalists
def journalist_following_summary(follower_to_followed_df):
    # Following count
    following_count_df = pd.DataFrame(follower_to_followed_df.followed_user_id.value_counts().rename('journalist_follower_count'))

    # Join with user summary
    journalist_following_summary_df = user_summary_df.join(following_count_df)
    journalist_following_summary_df.fillna(0, inplace=True)
    journalist_following_summary_df = journalist_following_summary_df.sort_values(['journalist_follower_count', 'followers_count'], ascending=False)
    return journalist_following_summary_df

# Gender of top journalists followed by beltway journalists
def top_journalist_followed_gender_summary(followed_summary_df, head=100):
    top_followed_summary_df = followed_summary_df.head(head)
    return pd.DataFrame({'count': top_followed_summary_df.gender.value_counts(), 
                  'percentage': top_followed_summary_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

# Fields for displaying journalist mention summaries
journalist_following_summary_fields = ['screen_name', 'name', 'organization', 'gender', 'followers_count', 'journalist_follower_count']


## Following analysis

### All followed by beltway journalists

In [109]:
# Following count
all_followed_df = pd.DataFrame(follower_to_followed_df.followed_user_id.value_counts().rename('following_count')).join(followed_screen_name_lookup_df)
all_followed_df.to_csv('output/all_followed_by_journalists.csv')
all_followed_df.head(25)

,following_count,screen_name
813286,1671,BarackObama
51241574,1629,AP
25073877,1613,realDonaldTrump
807095,1581,nytimes
2467791,1532,washingtonpost
1339835893,1531,HillaryClinton
818927131883356161,1522,PressSec
822215673812119553,1507,WhiteHouse
822215679726100480,1488,POTUS
9300262,1457,politico


### All beltway journalists followed by beltway journalists

In [110]:
follower_to_journalist_followed_summary_df = journalist_following_summary(follower_to_journalist_followed_df)
follower_to_journalist_followed_summary_df.to_csv('output/journalists_followed_by_journalists.csv')
follower_to_journalist_followed_summary_df[journalist_following_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,journalist_follower_count
user_id,,,,,,
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,"1,337.00"
50325797,chucktodd,"Todd, Chuck",NBC News,M,1781247,"1,258.00"
19107878,GlennThrush,"Thrush, Glenn H.",New York Times,M,308181,"1,116.00"
31127446,markknoller,"Knoller, Mark",CBS News,M,301474,"1,107.00"
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,"1,106.00"
61734492,Fahrenthold,"Fahrenthold, David",Washington Post,M,451778,"1,082.00"
18678924,jmartNYT,"Martin, Jonathan",New York Times,M,197322,"1,032.00"
39155029,mkraju,"Raju, Manu K.",CNN,M,88366,977.00
16930125,edatpost,"O’Keefe, Edward",Washington Post,M,58670,973.00


In [111]:
journalist_followed_gender_summary(follower_to_journalist_followed_df)


,count,percentage
M,174283,62.2%
F,106057,37.8%


#### Stats of beltway journalists followed by beltway journalists

In [112]:
follower_to_journalist_followed_summary_df[['journalist_follower_count']].describe()

,journalist_follower_count
count,"2,292.00"
mean,122.31
std,161.53
min,0.00
25%,26.00
50%,64.00
75%,145.00
max,"1,337.00"


#### Top 100 journalists followed by beltway journalists

In [113]:
top_journalist_followed_gender_summary(follower_to_journalist_followed_summary_df)

,count,percentage
M,76,76.0%
F,24,24.0%


### Female beltway journalists followed by beltway journalists

In [114]:
follower_to_female_journalist_followed_df = follower_to_journalist_followed_summary_df[follower_to_journalist_followed_summary_df.gender == 'F']
follower_to_female_journalist_followed_df.to_csv('output/female_journalists_followed_by_journalists.csv')
follower_to_female_journalist_followed_df[journalist_following_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,journalist_follower_count
user_id,,,,,,
89820928,mitchellreports,"Mitchell, Andrea",NBC News,F,1388543,909.00
108617810,DanaBashCNN,"Bash, Dana",CNN,F,281861,884.00
130945778,mollyesque,"Ball, Molly",The Atlantic,F,116857,877.00
12354832,kasie,"Hunt, Kasie",NBC News,F,187357,860.00
33919343,AshleyRParker,"Parker, Ashley",Washington Post,F,122382,777.00
28181835,jpaceDC,"Pace, Julie",Associated Press,F,46017,738.00
70511174,Hadas_Gold,"Gold, Hadas",Politico,F,45221,679.00
21307076,SusanPage,"Page, Susan",USA Today,F,48675,670.00
19186003,seungminkim,"Kim, Seung Min",Politico,F,33980,664.00


#### Stats of female beltway journalists followed by beltway journalists

In [115]:
follower_to_female_journalist_followed_df[['journalist_follower_count']].describe()

,journalist_follower_count
count,993.00
mean,106.80
std,131.81
min,0.00
25%,24.00
50%,59.00
75%,131.00
max,909.00


### Male beltway journalists followed by beltway journalists

In [116]:
follower_to_male_journalist_followed_df = follower_to_journalist_followed_summary_df[follower_to_journalist_followed_summary_df.gender == 'M']
follower_to_male_journalist_followed_df.to_csv('output/male_journalists_followed_by_journalists.csv')
follower_to_male_journalist_followed_df[journalist_following_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,journalist_follower_count
user_id,,,,,,
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,"1,337.00"
50325797,chucktodd,"Todd, Chuck",NBC News,M,1781247,"1,258.00"
19107878,GlennThrush,"Thrush, Glenn H.",New York Times,M,308181,"1,116.00"
31127446,markknoller,"Knoller, Mark",CBS News,M,301474,"1,107.00"
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,"1,106.00"
61734492,Fahrenthold,"Fahrenthold, David",Washington Post,M,451778,"1,082.00"
18678924,jmartNYT,"Martin, Jonathan",New York Times,M,197322,"1,032.00"
39155029,mkraju,"Raju, Manu K.",CNN,M,88366,977.00
16930125,edatpost,"O’Keefe, Edward",Washington Post,M,58670,973.00


#### Stats of male beltway journalists followed by beltway journalists

In [117]:
follower_to_male_journalist_followed_df[['journalist_follower_count']].describe()

,journalist_follower_count
count,"1,299.00"
mean,134.17
std,180.14
min,0.00
25%,28.00
50%,67.00
75%,156.00
max,"1,337.00"


### Beltway journalists followed by female beltway journalists

In [118]:
female_follower_to_journalist_followed_df = journalist_following_summary(follower_to_journalist_followed_df[follower_to_journalist_followed_df.gender == 'F'])
female_follower_to_journalist_followed_df.to_csv('output/journalists_followed_by_female_journalists.csv')
female_follower_to_journalist_followed_df[journalist_following_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,journalist_follower_count
user_id,,,,,,
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,619.00
50325797,chucktodd,"Todd, Chuck",NBC News,M,1781247,569.00
31127446,markknoller,"Knoller, Mark",CBS News,M,301474,505.00
19107878,GlennThrush,"Thrush, Glenn H.",New York Times,M,308181,490.00
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,484.00
61734492,Fahrenthold,"Fahrenthold, David",Washington Post,M,451778,474.00
18678924,jmartNYT,"Martin, Jonathan",New York Times,M,197322,445.00
16930125,edatpost,"O’Keefe, Edward",Washington Post,M,58670,444.00
89820928,mitchellreports,"Mitchell, Andrea",NBC News,F,1388543,441.00


In [119]:
journalist_followed_gender_summary(follower_to_journalist_followed_df[follower_to_journalist_followed_df.gender == 'F'])

,count,percentage
M,73950,62.0%
F,45300,38.0%


#### Top 100 journalists followed by female beltway journalists

In [120]:
top_journalist_followed_gender_summary(female_follower_to_journalist_followed_df)

,count,percentage
M,74,74.0%
F,26,26.0%


### Beltway journalists followed by male beltway journalists

In [121]:
male_follower_to_journalist_followed_df = journalist_following_summary(follower_to_journalist_followed_df[follower_to_journalist_followed_df.gender == 'M'])
male_follower_to_journalist_followed_df.to_csv('output/journalists_followed_by_male_journalists.csv')
male_follower_to_journalist_followed_df[journalist_following_summary_fields].head(25)

,screen_name,name,organization,gender,followers_count,journalist_follower_count
user_id,,,,,,
14529929,jaketapper,"Tapper, Jake",CNN,M,1305680,718.00
50325797,chucktodd,"Todd, Chuck",NBC News,M,1781247,689.00
19107878,GlennThrush,"Thrush, Glenn H.",New York Times,M,308181,626.00
13524182,daveweigel,"Weigel, David",Washington Post,M,332344,622.00
61734492,Fahrenthold,"Fahrenthold, David",Washington Post,M,451778,608.00
31127446,markknoller,"Knoller, Mark",CBS News,M,301474,602.00
18678924,jmartNYT,"Martin, Jonathan",New York Times,M,197322,587.00
39155029,mkraju,"Raju, Manu K.",CNN,M,88366,543.00
85131054,jeffzeleny,"Zeleny, Jeff",CNN,M,244114,535.00


In [122]:
journalist_followed_gender_summary(follower_to_journalist_followed_df[follower_to_journalist_followed_df.gender == 'M'])

,count,percentage
M,100333,62.3%
F,60757,37.7%


#### Top 100 journalists followed by male beltway journalists

In [123]:
top_journalist_followed_gender_summary(male_follower_to_journalist_followed_df)

,count,percentage
M,77,77.0%
F,23,23.0%


## Merge all

In [124]:
# Mention by all
user_merge_df = user_summary_df.join(journalists_mention_summary_df[['mention_count', 'mentioning_count']])
# Mention by female
user_merge_df = user_merge_df.join(journalists_mentioned_by_female_summary_df[['mention_count', 'mentioning_count']], rsuffix='_by_female')
# Mention by male
user_merge_df = user_merge_df.join(journalists_mentioned_by_male_summary_df[['mention_count', 'mentioning_count']], rsuffix='_by_male')
# Retweet by all
user_merge_df = user_merge_df.join(journalists_retweet_summary_df[['retweet_count', 'retweeting_count']])
# Retweet by female
user_merge_df = user_merge_df.join(journalists_retweeted_by_female_summary_df[['retweet_count', 'retweeting_count']], rsuffix='_by_female')
# Retweet by male
user_merge_df = user_merge_df.join(journalists_retweeted_by_male_summary_df[['retweet_count', 'retweeting_count']], rsuffix='_by_male')
# Reply by all
user_merge_df = user_merge_df.join(journalists_reply_summary_df[['reply_to_count', 'replying_count']])
# Reply by female
user_merge_df = user_merge_df.join(journalists_replied_to_by_female_summary_df[['reply_to_count', 'replying_count']], rsuffix='_by_female')
# Reply by male
user_merge_df = user_merge_df.join(journalists_replied_to_by_male_summary_df[['reply_to_count', 'replying_count']], rsuffix='_by_male')
# Follows all
user_merge_df = user_merge_df.join(follower_to_journalist_followed_summary_df[['journalist_follower_count']])
# Follows female
user_merge_df = user_merge_df.join(female_follower_to_journalist_followed_df[['journalist_follower_count']], rsuffix='_by_female')
# Follows male
user_merge_df = user_merge_df.join(male_follower_to_journalist_followed_df[['journalist_follower_count']], rsuffix='_by_male')
user_merge_df.fillna(0, inplace=True)
# Write to CSV
user_merge_df.to_csv('output/journalists_summary.csv')
user_merge_df.count()

screen_name                            2292
name                                   2292
organization                           2292
position                               2292
gender                                 2292
followers_count                        2292
following_count                        2292
tweet_count                            2292
user_created_at                        2292
verified                               2292
protected                              2292
original                               2292
quote                                  2292
reply                                  2292
retweet                                2292
tweets_in_dataset                      2292
mention_count                          2292
mentioning_count                       2292
mention_count_by_female                2292
mentioning_count_by_female             2292
mention_count_by_male                  2292
mentioning_count_by_male               2292
retweet_count                   

In [125]:
user_merge_df.head()

,screen_name,name,organization,position,gender,followers_count,following_count,tweet_count,user_created_at,verified,...,retweeting_count_by_male,reply_to_count,replying_count,reply_to_count_by_female,replying_count_by_female,reply_to_count_by_male,replying_count_by_male,journalist_follower_count,journalist_follower_count_by_female,journalist_follower_count_by_male
user_id,,,,,,,,,,,,,,,,,,,,,
23455653,abettel,"Bettelheim, Adriel",Politico,Health Care Editor,F,2664,1055,15990,Mon Mar 09 16:32:20 +0000 2009,True,...,16.00,3.00,3.00,0.00,0.00,3.00,3.00,179.00,80.00,99.00
33919343,AshleyRParker,"Parker, Ashley",Washington Post,White House Reporter,F,122382,2342,12433,Tue Apr 21 14:28:57 +0000 2009,True,...,172.00,26.00,18.00,4.00,4.00,22.00,14.00,777.00,339.00,438.00
18580432,b_fung,"Fung, Brian",Washington Post,Tech Reporter,M,16558,2062,44799,Sat Jan 03 15:15:57 +0000 2009,True,...,22.00,93.00,17.00,10.00,6.00,83.00,11.00,221.00,94.00,127.00
399225358,b_muzz,"Murray, Brendan",Bloomberg News,"Managing Editor, U.S. Economy",M,624,382,360,Thu Oct 27 05:34:05 +0000 2011,True,...,2.00,0.00,0.00,0.00,0.00,0.00,0.00,45.00,13.00,32.00
18834692,becca_milfeld,"Milfeld, Becca",Agence France-Presse,English Desk Editor and Journalist,F,483,993,1484,Sat Jan 10 13:58:43 +0000 2009,False,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.00,9.00,9.00


### Describe a row

In [126]:
user_merge_df.loc['407013776']

screen_name                                                 burgessev
name                                                 Everett, John B.
organization                                                 Politico
position                                       Congressional Reporter
gender                                                              M
followers_count                                                 31010
following_count                                                  1782
tweet_count                                                     27294
user_created_at                        Mon Nov 07 14:22:19 +0000 2011
verified                                                         True
protected                                                       False
original                                                       836.00
quote                                                          344.00
reply                                                          275.00
retweet             

* followers_count: The number of followers (including non-journalists)
* following_count: The number of friends, i.e., accounts that @burgessev is following including non-journalists)
* tweet_count: The number of lifetime tweets posted by @burgessev.
* original: The number of original tweets posted by @burgessev in this dataset.
* quote: The number of quote tweets posted by @burgessev in this dataset.
* reply: The number of reply tweets posted by @burgessev in this dataset.
* retweet: The number of retweets posted by @burgessev in this dataset.
* tweets_in_dataset: The total number of tweets posted by @burgessev in this dataset. Equal to original + quote + reply + retweet
* mention_count: The number of mentions of @burgessev by journalists. Equal to mention_count_by_female + mention_count_by_male.
* mentioning_count: The number of journalists that mentioned @burgessev. Equal to mentioning_count_by_female + mentioning_count_by_male.
* mention_count_by_female / mention_count_by_male: The number of mentions of @burgessev by female / male journalists.
* mentioning_count_by_female / mentioning_count_by_male: The number of female / male journalists that mentioned @burgessev.
* retweet_count: The number of retweets of @burgessev by journalists. Equal to retweet_count_by_female + retweet_count_by_male.
* retweeting_count: The number of journalists that retweeted @burgessev. Equal to retweeting_count_by_female + retweeting_count_by_male.
* retweet_count_by_female / retweet_count_by_male: The number of retweets of @burgessev by female / male journalists.
* retweeting_count_by_female / retweeting_count_by_male: The number of female / male journalists that retweeted @burgessev.
* reply_count: The number of replies to @burgessev by journalists. Equal to reply_count_by_female + reply_count_by_male.
* replying_count: The number of journalists that replied to @burgessev. Equal to replying_count_by_female + replying_count_by_male.
* reply_count_by_female / reply_count_by_male: The number of replies to @burgessev by female / male journalists.
* replying_count_by_female / replying_count_by_male: The number of female / male journalists that replied to @burgessev.
* journalist_follower_count: The number of journalists following @burgessev. Equal to journalist_follower_count_by_female + journalist_follower_count_by_male.
* journalist_follower_count_by_female / journalist_follower_count_by_female: The number of female / male journalists following @burgessev.